In [12]:
import json

response = {
  "id": "msg_016nRp4M2fLPv6V8mnLdQtsL",
  "type": "message",
  "role": "assistant",
  "model": "claude-opus-4-6",
  "content": [
    {
      "type": "text",
      "text": "# 🧪 Prompt Engineering Lab with Gemini API — Enhanced Edition\n\n## 📚 Lab Overview & Learning Objectives\n\nLab นี้ออกแบบมาเพื่อให้ผู้เรียนได้ฝึกฝนเทคนิค Prompt Engineering ขั้นสูงผ่าน Gemini API โดยเริ่มจากพื้นฐานไปจนถึงเทคนิคขั้นสูง พร้อมระบบ Evaluation ที่ครบถ้วน\n\n---\n\n## 🗺️ Lab Roadmap — ภาพรวมเส้นทางการเรียนรู้\n\n```\n╔══════════════════════════════════════════════════════════════════════════════════╗\n║                    🗺️ PROMPT ENGINEERING LEARNING PATH                         ║\n╠══════════════════════════════════════════════════════════════════════════════════╣\n║                                                                                ║\n║  🟢 FOUNDATION (Part 0-2)                                                      ║\n║  ┌─────────────┐    ┌─────────────────┐    ┌──────────────────┐                ║\n║  │  Part 0     │───▶│   Part 1        │───▶│   Part 2         │                ║\n║  │  Setup &    │    │   Few-shot      │    │   Chain-of-      │                ║\n║  │  Connection │    │   Prompting ⭐   │    │   Thought ⭐⭐    │                ║\n║  └─────────────┘    └─────────────────┘    └────────┬─────────┘                ║\n║                                                      │                          ║\n║  🟡 INTERMEDIATE (Part 3-4)                          ▼                          ║\n║  ┌─────────────────────────┐    ┌──────────────────────────┐                   ║\n║  │   Part 3                │    │   Part 4                 │                   ║\n║  │   Self-Consistency      │◄──▶│   Prompt Chaining        │                   ║\n║  │   CoT ⭐⭐⭐              │    │   ⭐⭐⭐                    │                   ║\n║  └────────────┬────────────┘    └────────────┬─────────────┘                   ║\n║               │                               │                                 ║\n║  🟠 ADVANCED (Part 5-8)     ▼               ▼                                 ║\n║  ┌──────────────────┐  ┌──────────────────┐  ┌──────────────────┐              ║\n║  │   Part 5         │  │   Part 6         │  │   Part 7         │              ║\n║  │   Reflection     │  │   Tree-of-       │  │   Self-Feedback  │              ║\n║  │   ⭐⭐⭐⭐          │  │   Thought ⭐⭐⭐⭐⭐ │  │   ⭐⭐⭐⭐          │              ║\n║  └────────┬─────────┘  └────────┬─────────┘  └────────┬─────────┘              ║\n║           │                     │                      │                        ║\n║           └─────────────────────┼──────────────────────┘                        ║\n║                                 ▼                                               ║\n║                      ┌──────────────────┐                                       ║\n║                      │   Part 8         │                                       ║\n║                      │   Self-Critique  │                                       ║\n║                      │   ⭐⭐⭐⭐⭐         │                                       ║\n║                      └────────┬─────────┘                                       ║\n║                               │                                                 ║\n║  🔴 EVALUATION (Part 9-10)    ▼                                                ║\n║  ┌──────────────────────────────────────────────────┐                           ║\n║  │   Part 9-10: Evaluation Methods                  │                           ║\n║  │   • LLM-as-a-Judge    • Reference-free           │                           ║\n║  │   • Pairwise          • BLEU/ROUGE               │                           ║\n║  │   • Rubric-based      • Semantic Similarity      │                           ║\n║  │   • Hallucination Detection                      │                           ║\n║  └────────────────────────┬─────────────────────────┘                           ║\n║                           ▼                                                     ║\n║  🏆 CAPSTONE (Part 11)                                                          ║\n║  ┌──────────────────────────────────────────────────┐                           ║\n║  │   Part 11: Final Challenge                       │                           ║\n║  │   Combined All Techniques + Full Evaluation      │                           ║\n║  └──────────────────────────────────────────────────┘                           ║\n║                                                                                ║\n╚══════════════════════════════════════════════════════════════════════════════════╝\n```\n\n---\n\n## 🧠 Prompt Engineering — ทำไมถึงสำคัญ?\n\n### ปัญหาที่ Prompt Engineering แก้ได้\n\n```\n╔═══════════════════════════════════════════════════════════════════╗\n║              🎯 WHY PROMPT ENGINEERING MATTERS                   ║\n╠═══════════════════════════════════════════════════════════════════╣\n║                                                                   ║\n║  ❌ ปัญหาเมื่อไม่ใช้เทคนิค          ✅ เมื่อใช้เทคนิคที่ถูกต้อง  ║\n║  ┌─────────────────────┐         ┌─────────────────────────┐     ║\n║  │ • คำตอบไม่ตรงประเด็น │         │ • คำตอบแม่นยำ ตรงจุด    │     ║\n║  │ • ขาดความสม่ำเสมอ   │   ───▶  │ • Output สม่ำเสมอ       │     ║\n║  │ • มี Hallucination  │         │ • ลด Hallucination      │     ║\n║  │ • ขาดเหตุผลรองรับ   │         │ • มี Reasoning ชัดเจน   │     ║\n║  │ • ไม่สามารถทำงาน    │         │ • ทำงานซับซ้อนได้       │     ║\n║  │   ซับซ้อนได้         │         │ • ประเมินคุณภาพได้      │     ║\n║  └─────────────────────┘         └─────────────────────────┘     ║\n║                                                                   ║\n║  📊 ผลวิจัย: Few-shot + CoT ช่วยเพิ่มความถูกต้องได้ 40-70%      ║\n║     ในงาน Reasoning (Wei et al., 2022)                           ║\n╚═══════════════════════════════════════════════════════════════════╝\n```\n\n### Prompt Engineering Taxonomy — แผนผังเทคนิคทั้งหมด\n\n```\n                          🧠 Prompt Engineering Techniques\n                                       │\n                 ┌─────────────────────┼─────────────────────┐\n                 │                     │                     │\n          📝 Input Design       🔄 Iterative          📊 Evaluation\n                 │              Refinement                   │\n         ┌───────┼───────┐     ┌───┼───┐          ┌────────┼────────┐\n         │       │       │     │       │          │        │        │\n     Zero-shot  Few-  System  Reflection Self-  LLM-as  Reference  Pairwise\n               shot  Prompt           Critique  Judge    -free     Compare\n         │       │               │       │\n    ┌────┤  ┌────┤          ┌────┤  ┌────┤\n    │    │  │    │          │    │  │    │\n   CoT  ToT One- Multi-  Multi  Self  Adversarial\n            shot  shot   round  Feedback  Critique\n    │\n    ├── Few-shot CoT\n    ├── Self-Consistency CoT\n    └── Prompt Chaining (Decomposition)\n```\n\n---\n\n## 📖 ทฤษฎีพื้นฐาน: LLM ทำงานอย่างไร?\n\n### LLM Processing Pipeline\n\n```\n╔═══════════════════════════════════════════════════════════════╗\n║              🔄 LLM PROCESSING PIPELINE                      ║\n╠═══════════════════════════════════════════════════════════════╣\n║                                                               ║\n║  📝 User Prompt                                               ║\n║      │                                                        ║\n║      ▼                                                        ║\n║  ┌──────────────────┐                                         ║\n║  │   Tokenization   │  \"สวัสดี\" → [token_1, token_2, ...]    ║\n║  └────────┬─────────┘                                         ║\n║           ▼                                                   ║\n║  ┌──────────────────┐                                         ║\n║  │   Embedding      │  tokens → vector representations       ║\n║  └────────┬─────────┘                                         ║\n║           ▼                                                   ║\n║  ┌──────────────────┐                                         ║\n║  │  Transformer     │  Self-Attention + Feed-Forward          ║\n║  │  Layers (×N)     │  Context understanding                  ║\n║  └────────┬─────────┘                                         ║\n║           ▼                                                   ║\n║  ┌──────────────────┐                                         ║\n║  │  Output Layer    │  Probability distribution               ║\n║  │  (Softmax)       │  over vocabulary                        ║\n║  └────────┬─────────┘                                         ║\n║           ▼                                                   ║\n║  ┌──────────────────┐                                         ║\n║  │  Sampling /      │  temperature, top-p, top-k              ║\n║  │  Decoding        │  ← Prompt Engineering affects here!     ║\n║  └────────┬─────────┘                                         ║\n║           ▼                                                   ║\n║  📄 Generated Response                                        ║\n║                                                               ║\n║  💡 KEY INSIGHT:                                              ║\n║  Prompt Engineering = การออกแบบ Input ให้ LLM สร้าง          ║\n║  probability distribution ที่นำไปสู่ output ที่ต้องการ        ║\n╚═══════════════════════════════════════════════════════════════╝\n```\n\n### Temperature & Sampling — ผลต่อ Output\n\n```\n╔══════════════════════════════════════════════════════════════╗\n║          🌡️ TEMPERATURE EFFECT ON OUTPUT                     ║\n╠══════════════════════════════════════════════════════════════╣\n║                                                              ║\n║  Temperature = 0.0 (Deterministic)                           ║\n║  ┌─────────────────────────────────────────────┐            ║\n║  │ █████████████████████████░░░░░░░░░░░░░░░░░░ │ \"แมว\"      ║\n║  │ ██░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░ │ \"สุนัข\"    ║\n║  │ ░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░ │ \"นก\"       ║\n║  └─────────────────────────────────────────────┘            ║\n║  → เลือก \"แมว\" เสมอ (ผลลัพธ์เหมือนเดิมทุกครั้ง)             ║\n║  → เหมาะกับ: Factual Q&A, Math, Classification              ║\n║                                                              ║\n║  Temperature = 0.7 (Balanced)                                ║\n║  ┌─────────────────────────────────────────────┐            ║\n║  │ █████████████████░░░░░░░░░░░░░░░░░░░░░░░░░░ │ \"แมว\"      ║\n║  │ ██████████░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░ │ \"สุนัข\"    ║\n║  │ █████░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░ │ \"นก\"       ║\n║  └─────────────────────────────────────────────┘            ║\n║  → มีโอกาสเลือกทุกตัว (น้ำหนักตามความน่าจะเป็น)             ║\n║  → เหมาะกับ: Creative writing, Brainstorming                ║\n║                                                              ║\n║  Temperature = 1.5 (Very Creative)                           ║\n║  ┌─────────────────────────────────────────────┐            ║\n║  │ █████████░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░ │ \"แมว\"      ║\n║  │ ████████░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░ │ \"สุนัข\"    ║\n║  │ ███████░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░ │ \"นก\"       ║\n║  └─────────────────────────────────────────────┘            ║\n║  → ความน่าจะเป็นเกือบเท่ากัน (สุ่มมาก อาจไม่ make sense)   ║\n║  → ⚠️ ระวัง: อาจได้ output ที่ไม่สอดคล้อง                    ║\n╚══════════════════════════════════════════════════════════════╝\n```\n\n---\n\n```python\n#%% [markdown]\n# # 🧪 Prompt Engineering Advanced Lab with Gemini API\n# # Enhanced Edition — with Extended Theory, Infographics & Evaluation\n# \n# ## 📋 Lab Description\n# Lab นี้จะพาคุณเรียนรู้เทคนิค Prompt Engineering ขั้นสูงทั้ง 8 เทคนิค:\n# 1. Few-shot Prompting\n# 2. Chain-of-Thought (CoT)\n# 3. Self-consistency Chain-of-Thought\n# 4. Prompt Chaining\n# 5. Reflection\n# 6. Tree-of-Thought (ToT)\n# 7. Self-feedback\n# 8. Self-critique\n# \n# พร้อมระบบ Evaluation ขั้นสูง:\n# - LLM-as-a-Judge (Single & Pairwise)\n# - Reference-free Evaluation\n# - Rubric-based Scoring\n# - BLEU / ROUGE Score (Automated Metrics)\n# - Semantic Similarity\n# - Hallucination Detection\n# - Multi-dimensional Evaluation Framework\n# \n# ## 🎯 Learning Outcomes\n# - เข้าใจและประยุกต์ใช้เทคนิค Prompt Engineering ขั้นสูง\n# - เปรียบเทียบข้อดี-ข้อเสียของแต่ละเทคนิค\n# - สามารถเลือกเทคนิคที่เหมาะสมกับปัญหาต่างๆ\n# - ประเมินคุณภาพ Output ด้วยวิธีการที่เป็นระบบและหลากหลาย\n\n#%% [markdown]\n# ---\n# # 🔧 Part 0: Environment Setup\n# ---\n# \n# ## 📖 ทฤษฎีพื้นฐาน: Gemini API\n# \n# **Gemini** คือ Large Language Model (LLM) จาก Google DeepMind\n# ที่รองรับทั้ง text, image, audio, video (Multimodal)\n# \n# ### API Parameters ที่ควรรู้:\n# ```\n# ┌────────────────────┬───────────────────────────────────────┐\n# │ Parameter          │ คำอธิบาย                              │\n# ├────────────────────┼───────────────────────────────────────┤\n# │ temperature        │ ความสุ่มของ output (0.0 - 2.0)       │\n# │                    │ 0.0 = deterministic, 1.0+ = creative │\n# │ top_p              │ Nucleus sampling threshold            │\n# │ top_k              │ จำนวน tokens ที่พิจารณา              │\n# │ max_output_tokens  │ จำนวน tokens สูงสุดใน response       │\n# │ stop_sequences     │ token ที่หยุดสร้าง output            │\n# └────────────────────┴───────────────────────────────────────┘\n# ```\n# \n# ### เมื่อไหร่ใช้ Parameter ใด:\n# ```\n# 📊 Task ต้องการความแม่นยำ (Math, Classification):\n#    → temperature = 0.0-0.2, top_p = 0.8\n# \n# ✍️ Task ต้องการความคิดสร้างสรรค์ (Writing, Brainstorming):\n#    → temperature = 0.7-1.0, top_p = 0.95\n# \n# 🔄 Task ต้องการความหลากหลาย (Self-Consistency):\n#    → temperature = 0.7-1.0 (ต้องการ diverse outputs)\n# ```\n\n#%%\n# ติดตั้ง Library ที่จำเป็น\n# !pip install google-genai pandas numpy\n\n#%%\n# Import Libraries\nfrom google import genai\nfrom google.genai import types\nimport json\nimport time\nimport re\nimport math\nfrom typing import List, Dict, Any, Tuple, Optional\nfrom collections import Counter\n\n#%%\n# กำหนด API Key และ Model\nAPI_KEY = \"YOUR_API_KEY_HERE\"  # ใส่ API Key ของคุณ\nMODEL_NAME = \"gemini-2.0-flash\"\n\n# สร้าง Client\nclient = genai.Client(api_key=API_KEY)\n\n#%%\n# ทดสอบ Connection\nresponse = client.models.generate_content(\n    model=MODEL_NAME,\n    contents=\"สวัสดี! ตอบสั้นๆ ว่าคุณพร้อมทำงานแล้ว\"\n)\nprint(\"✅ Connection Test:\", response.text)\n\n#%%\n# === Helper Function สำหรับใช้ตลอดทั้ง Lab ===\n\ndef call_gemini(prompt: str, temperature: float = 0.0, max_tokens: int = 2048) -> str:\n    \"\"\"Helper function สำหรับเรียก Gemini API\"\"\"\n    response = client.models.generate_content(\n        model=MODEL_NAME,\n        contents=prompt,\n        config=types.GenerateContentConfig(\n            temperature=temperature,\n            max_output_tokens=max_tokens,\n        )\n    )\n    return response.text\n\ndef print_section(title: str, content: str, emoji: str = \"🔹\"):\n    \"\"\"Helper function สำหรับแสดงผลแบบสวยงาม\"\"\"\n    print(f\"\\n{emoji} {title}\")\n    print(\"─\" * 50)\n    print(content)\n    print(\"─\" * 50)\n\n#%% [markdown]\n# ---\n# # 📘 Part 1: Few-shot Prompting\n# **ระดับความยาก: ⭐ (เริ่มต้น)**\n# \n# ---\n# ## 📖 ทฤษฎีเชิงลึก\n# \n# **Few-shot Prompting** คือเทคนิคที่ให้ตัวอย่าง (examples) จำนวนหนึ่งใน prompt \n# เพื่อช่วยให้ LLM เข้าใจรูปแบบและ pattern ที่ต้องการ\n# \n# ### 📊 Infographic: สเปกตรัมของ Shot Types\n# \n# ```\n# ┌──────────────────────────────────────────────────────────────────────┐\n# │                    📊 SHOT SPECTRUM                                  │\n# │                                                                      │\n# │  Zero-shot        One-shot        Few-shot        Many-shot         │\n# │  (0 examples)     (1 example)     (2-5 examples)  (6+ examples)    │\n# │                                                                      │\n# │  ○────────────────●────────────────●────────────────●──────▶        │\n# │                                                                      │\n# │  Flexibility ◄─────────────────────────────────── Consistency       │\n# │  HIGH                                              HIGH              │\n# │                                                                      │\n# │  Best for:        Best for:       Best for:        Best for:        │\n# │  - Simple tasks   - Quick demo    - Complex format - Very specific  │\n# │  - General Q&A    - Basic pattern - Consistent     - Custom domain  │\n# │  - Model already  - When data     - Classification - Edge cases     │\n# │    knows well       is limited    - Transformation   matter         │\n# └──────────────────────────────────────────────────────────────────────┘\n# ```\n# \n# ### 📐 หลักการออกแบบตัวอย่างที่ดี\n# \n# ```\n# ┌─────────────────────────────────────────────────┐\n# │         🎯 GOOD EXAMPLE DESIGN PRINCIPLES       │\n# │                                                   │\n# │  1. ✅ REPRESENTATIVE (เป็นตัวแทนที่ดี)           │\n# │     → เลือกตัวอย่างที่ครอบคลุม cases สำคัญ       │\n# │                                                   │\n# │  2. ✅ DIVERSE (หลากหลาย)                         │\n# │     → ไม่ใช้ตัวอย่างที่คล้ายกันหมด               │\n# │                                                   │\n# │  3. ✅ FORMATTED (มี format ชัดเจน)               │\n# │     → Input/Output แยกกันชัด                      │\n# │                                                   │\n# │  4. ✅ BALANCED (สมดุล)                            │\n# │     → ถ้าเป็น classification ครบทุก class         │\n# │                                                   │\n# │  5. ❌ AVOID: ตัวอย่างที่ขัดแย้งกัน               │\n# │  6. ❌ AVOID: ตัวอย่างที่ยาวเกินไป                │\n# │  7. ❌ AVOID: ตัวอย่างที่ ambiguous                │\n# └─────────────────────────────────────────────────┘\n# ```\n# \n# ### 🔬 งานวิจัยที่เกี่ยวข้อง\n# - **Brown et al. (2020)** — \"Language Models are Few-Shot Learners\" (GPT-3 Paper)\n#   พบว่า LLM สามารถเรียนรู้จากตัวอย่างในบริบทได้โดยไม่ต้อง fine-tune\n# - **Min et al. (2022)** — \"Rethinking the Role of Demonstrations\"\n#   พบว่า *format* ของตัวอย่างสำคัญกว่า *ความถูกต้อง* ของ label\n\n#%%\n# === Lab 1.1: Zero-shot vs Few-shot Comparison ===\n# Task: จำแนกความรู้สึก (Sentiment Analysis) ของรีวิวร้านอาหาร\n\n# Zero-shot\nzero_shot_prompt = \"\"\"\nจำแนกความรู้สึกของรีวิวต่อไปนี้เป็น: บวก, ลบ, หรือ กลาง\n\nรีวิว: \"อาหารรสชาติดีมาก แต่รอนานไปหน่อย\"\nความรู้สึก:\n\"\"\"\n\nresponse_zero = call_gemini(zero_shot_prompt)\nprint_section(\"Zero-shot Result\", response_zero)\n\n#%%\n# Few-shot (3 examples)\nfew_shot_prompt = \"\"\"\nจำแนกความรู้สึกของรีวิวร้านอาหารเป็น: บวก, ลบ, หรือ กลาง\n\nตัวอย่าง 1:\nรีวิว: \"อร่อยมากค่ะ บริการดีเยี่ยม จะกลับมาอีกแน่นอน!\"\nความรู้สึก: บวก\n\nตัวอย่าง 2:\nรีวิว: \"ผิดหวังมาก อาหารเย็น พนักงานไม่สุภาพ\"\nความรู้สึก: ลบ\n\nตัวอย่าง 3:\nรีวิว: \"อาหารพอใช้ได้ ราคาก็โอเค ไม่ได้โดดเด่นอะไร\"\nความรู้สึก: กลาง\n\nตอนนี้จำแนกรีวิวนี้:\nรีวิว: \"อาหารรสชาติดีมาก แต่รอนานไปหน่อย\"\nความรู้สึก:\n\"\"\"\n\nresponse_few = call_gemini(few_shot_prompt)\nprint_section(\"Few-shot Result\", response_few)\n\n#%%\n# === Lab 1.2: Few-shot สำหรับ Data Transformation ===\n# Task: แปลงที่อยู่ไทยเป็น JSON format\n\nfew_shot_address = \"\"\"\nแปลงที่อยู่ภาษาไทยเป็น JSON format\n\nตัวอย่าง 1:\nInput: \"123/45 ซอยสุขุมวิท 21 ถนนสุขุมวิท แขวงคลองเตยเหนือ เขตวัฒนา กรุงเทพฯ 10110\"\nOutput: {\n  \"house_number\": \"123/45\",\n  \"soi\": \"สุขุมวิท 21\",\n  \"road\": \"สุขุมวิท\",\n  \"subdistrict\": \"คลองเตยเหนือ\",\n  \"district\": \"วัฒนา\",\n  \"province\": \"กรุงเทพมหานคร\",\n  \"postal_code\": \"10110\"\n}\n\nตัวอย่าง 2:\nInput: \"89 หมู่ 5 ตำบลบางพลี อำเภอบางพลี จังหวัดสมุทรปราการ 10540\"\nOutput: {\n  \"house_number\": \"89\",\n  \"moo\": \"5\",\n  \"subdistrict\": \"บางพลี\",\n  \"district\": \"บางพลี\",\n  \"province\": \"สมุทรปราการ\",\n  \"postal_code\": \"10540\"\n}\n\nตอนนี้แปลงที่อยู่นี้:\nInput: \"456/7 ถนนเพชรบุรีตัดใหม่ แขวงบางกะปิ เขตห้วยขวาง กรุงเทพฯ 10310\"\nOutput:\n\"\"\"\n\nresponse = call_gemini(few_shot_address)\nprint_section(\"Address Transformation Result\", response)\n\n#%%\n# === Lab 1.3: Few-shot Sensitivity Analysis ===\n# ทดลองเปลี่ยนจำนวน examples และดูผลกระทบ\n\ndef test_few_shot_sensitivity(test_input: str, examples: list, test_cases: list = [0, 1, 2, 3]):\n    \"\"\"\n    ทดสอบว่าจำนวน examples มีผลต่อคุณภาพ output อย่างไร\n    \"\"\"\n    results = {}\n    \n    for n in test_cases:\n        if n == 0:\n            prompt = f\"\"\"จำแนกความรู้สึกของรีวิวเป็น: บวก, ลบ, หรือ กลาง (ตอบแค่คำเดียว)\n\nรีวิว: \"{test_input}\"\nความรู้สึก:\"\"\"\n        else:\n            selected_examples = examples[:n]\n            examples_text = \"\\n\\n\".join(selected_examples)\n            prompt = f\"\"\"จำแนกความรู้สึกของรีวิวเป็น: บวก, ลบ, หรือ กลาง (ตอบแค่คำเดียว)\n\n{examples_text}\n\nรีวิว: \"{test_input}\"\nความรู้สึก:\"\"\"\n        \n        response = call_gemini(prompt)\n        results[f\"{n}-shot\"] = response.strip()\n        print(f\"  {n}-shot → {response.strip()}\")\n    \n    return results\n\nexamples = [\n    'รีวิว: \"อร่อยสุดๆ แนะนำเลย!\"\\nความรู้สึก: บวก',\n    'รีวิว: \"แย่มาก ไม่มาอีกแล้ว\"\\nความรู้สึก: ลบ',\n    'รีวิว: \"ก็ธรรมดา ไม่ได้รู้สึกอะไรเป็นพิเศษ\"\\nความรู้สึก: กลาง',\n]\n\nprint(\"📊 Few-shot Sensitivity Analysis:\")\nprint(\"=\" * 40)\nsensitivity_results = test_few_shot_sensitivity(\n    \"อาหารรสชาติดี แต่พนักงานไม่ค่อยยิ้มแย้ม ราคาก็กลางๆ\",\n    examples\n)\n\n#%% [markdown]\n# ### 📝 แบบฝึกหัด 1: Few-shot Prompting\n# \n# **โจทย์**: สร้าง Few-shot prompt สำหรับแปลงชื่อผลิตภัณฑ์เป็น SKU code\n# - ตัวอย่าง: \"เสื้อยืดสีขาว ไซส์ L\" → \"TSH-WHT-L\"\n# - สร้างอย่างน้อย 3 ตัวอย่าง\n# - ทดสอบกับ input ใหม่\n# \n# **Hints:**\n# ```\n# SKU Pattern: [CATEGORY]-[COLOR]-[SIZE]\n# Categories: TSH=เสื้อยืด, PLO=เสื้อโปโล, JNS=กางเกงยีนส์\n# Colors: WHT=ขาว, BLK=ดำ, BLU=น้ำเงิน, RED=แดง\n# Sizes: S, M, L, XL\n# ```\n\n#%%\n# === แบบฝึกหัด 1: เขียน Code ของคุณที่นี่ ===\n\n# TODO: สร้าง few-shot prompt สำหรับ SKU generation\nfew_shot_sku_prompt = \"\"\"\n# เขียน prompt ของคุณที่นี่\n\n\n\"\"\"\n\n# response = call_gemini(few_shot_sku_prompt)\n# print(response)\n\n#%% [markdown]\n# ---\n# # 📘 Part 2: Chain-of-Thought (CoT)\n# **ระดับความยาก: ⭐⭐ (กลาง)**\n# \n# ---\n# ## 📖 ทฤษฎีเชิงลึก\n# \n# **Chain-of-Thought (CoT)** คือเทคนิคที่กระตุ้นให้ LLM แสดงขั้นตอนการคิด\n# อย่างเป็นลำดับก่อนให้คำตอบสุดท้าย ช่วยปรับปรุงความถูกต้องในงานที่ต้องใช้เหตุผล\n# \n# ### 📊 Infographic: CoT ทำงานอย่างไร\n# \n# ```\n# ╔═══════════════════════════════════════════════════════════════════╗\n# ║                  🧠 CHAIN-OF-THOUGHT MECHANISM                   ║\n# ╠═══════════════════════════════════════════════════════════════════╣\n# ║                                                                   ║\n# ║  ❌ Standard Prompting (Direct Answer):                           ║\n# ║  ┌──────────┐         ┌──────────┐                               ║\n# ║  │ Question │ ──────▶ │  Answer  │   \"Black Box\" ❓               ║\n# ║  └──────────┘         └──────────┘                               ║\n# ║                                                                   ║\n# ║  ✅ Chain-of-Thought Prompting:                                   ║\n# ║  ┌──────────┐   ┌──────┐   ┌──────┐   ┌──────┐   ┌──────────┐  ║\n# ║  │ Question │──▶│Step 1│──▶│Step 2│──▶│Step 3│──▶│  Answer  │  ║\n# ║  └──────────┘   └──────┘   └──────┘   └──────┘   └──────────┘  ║\n# ║                     │          │          │                       ║\n# ║                 Decompose   Reason    Synthesize                 ║\n# ║                 (แยกส่วน)  (ให้เหตุผล)  (สรุป)                    ║\n# ║                                                                   ║\n# ║  📊 Performance Gain (Wei et al., 2022):                         ║\n# ║  ┌────────────────────────────────────────────┐                  ║\n# ║  │ Task          │ Standard │  CoT   │ Gain   │                  ║\n# ║  │───────────────│──────────│────────│────────│                  ║\n# ║  │ GSM8K (Math)  │  17.7%   │ 57.1%  │ +39.4% │                  ║\n# ║  │ SVAMP (Math)  │  63.2%   │ 79.0%  │ +15.8% │                  ║\n# ║  │ CommonsenseQA │  73.5%   │ 80.5%  │ +7.0%  │                  ║\n# ║  └────────────────────────────────────────────┘                  ║\n# ╚═══════════════════════════════════════════════════════════════════╝\n# ```\n# \n# ### 🔑 Trigger Phrases ที่ใช้กระตุ้น CoT\n# \n# ```\n# ┌──────────────────────────────────────────────┐\n# │         🔑 CoT TRIGGER PHRASES               │\n# │                                                │\n# │  English:                                      │\n# │  • \"Let's think step by step\"                  │\n# │  • \"Show your work/reasoning\"                  │\n# │  • \"Explain your thought process\"              │\n# │  • \"Break this down into steps\"                │\n# │                                                │\n# │  ภาษาไทย:                                      │\n# │  • \"คิดทีละขั้นตอน\"                             │\n# │  • \"แสดงวิธีคิดทุกขั้นตอน\"                      │\n# │  • \"อธิบายกระบวนการคิดของคุณ\"                   │\n# │  • \"แยกปัญหาเป็นส่วนย่อย\"                       │\n# │                                                │\n# │  💡 TIP: ยิ่งเจาะจง ยิ่งดี                      │\n# │  แทนที่: \"คิดทีละขั้นตอน\"                       │\n# │  ใช้:   \"คิดทีละขั้นตอน: 1)ระบุข้อมูล          │\n# │          2)ตั้งสมการ 3)คำนวณ 4)สรุป\"            │\n# └──────────────────────────────────────────────┘\n# ```\n# \n# ### CoT Variants:\n# ```\n# CoT Family:\n# ├── Zero-shot CoT: \"Let's think step by step\" (ไม่มีตัวอย่าง)\n# ├── Few-shot CoT: มีตัวอย่างพร้อม reasoning steps\n# ├── Self-Consistency CoT: หลาย paths → majority vote (Part 3)\n# ├── Auto-CoT: ให้ LLM สร้างตัวอย่าง CoT เอง\n# └── Multimodal CoT: CoT กับ image + text\n# ```\n\n#%%\n# === Lab 2.1: Basic Chain-of-Thought ===\n# Task: แก้ปัญหาคณิตศาสตร์\n\n# ❌ Without CoT\nno_cot_prompt = \"\"\"\nร้านค้าขายเสื้อ 3 ตัว ราคาตัวละ 250 บาท และกางเกง 2 ตัว ราคาตัวละ 350 บาท\nถ้าลูกค้าได้ส่วนลด 10% รวมทั้งหมดลูกค้าต้องจ่ายเท่าไหร่?\n\"\"\"\n\nresponse_no_cot = call_gemini(no_cot_prompt)\nprint_section(\"Without CoT\", response_no_cot, \"❌\")\n\n#%%\n# ✅ With CoT\ncot_prompt = \"\"\"\nร้านค้าขายเสื้อ 3 ตัว ราคาตัวละ 250 บาท และกางเกง 2 ตัว ราคาตัวละ 350 บาท\nถ้าลูกค้าได้ส่วนลด 10% รวมทั้งหมดลูกค้าต้องจ่ายเท่าไหร่?\n\nกรุณาคิดทีละขั้นตอนอย่างละเอียด แสดงวิธีคิดทุกขั้นตอน\n\"\"\"\n\nresponse_cot = call_gemini(cot_prompt)\nprint_section(\"With CoT\", response_cot, \"✅\")\n\n#%%\n# === Lab 2.2: CoT สำหรับ Logical Reasoning ===\n# Task: วิเคราะห์ปัญหาทางธุรกิจ\n\ncot_business_prompt = \"\"\"\nบริษัทมีข้อมูลดังนี้:\n- ยอดขายเดือนที่แล้ว: 1,000,000 บาท\n- ต้นทุนสินค้า: 60% ของยอดขาย\n- ค่าใช้จ่ายคงที่: 250,000 บาท\n- เป้าหมายกำไรสุทธิเดือนนี้: เพิ่มขึ้น 20%\n\nคำถาม: บริษัทต้องเพิ่มยอดขายเท่าไหร่ (สมมติต้นทุนและค่าใช้จ่ายคงที่เท่าเดิม)?\n\nกรุณาวิเคราะห์ทีละขั้นตอน:\n1. คำนวณกำไรปัจจุบัน\n2. คำนวณเป้าหมายกำไรใหม่\n3. คำนวณยอดขายที่ต้องการ\n4. สรุปผล\n\"\"\"\n\nresponse = call_gemini(cot_business_prompt)\nprint_section(\"Business Analysis with CoT\", response)\n\n#%%\n# === Lab 2.3: Few-shot CoT ===\n# รวม Few-shot กับ CoT เพื่อผลลัพธ์ที่ดียิ่งขึ้น\n\nfew_shot_cot_prompt = \"\"\"\nแก้ปัญหาคณิตศาสตร์โดยแสดงวิธีคิดทีละขั้นตอน\n\nตัวอย่าง 1:\nโจทย์: มีแอปเปิ้ล 5 ลูก ซื้อเพิ่มมา 3 ลูก แล้วแบ่งให้เพื่อนไป 2 ลูก เหลือกี่ลูก?\nวิธีคิด:\n- ขั้นตอน 1: มีแอปเปิ้ลเริ่มต้น = 5 ลูก\n- ขั้นตอน 2: ซื้อเพิ่ม = 5 + 3 = 8 ลูก\n- ขั้นตอน 3: แบ่งให้เพื่อน = 8 - 2 = 6 ลูก\nคำตอบ: 6 ลูก\n\nตัวอย่าง 2:\nโจทย์: รถวิ่งด้วยความเร็ว 60 กม./ชม. ใช้เวลา 2.5 ชั่วโมง วิ่งได้ระยะทางเท่าไหร่?\nวิธีคิด:\n- ขั้นตอน 1: ใช้สูตร ระยะทาง = ความเร็ว × เวลา\n- ขั้นตอน 2: แทนค่า = 60 × 2.5\n- ขั้นตอน 3: คำนวณ = 150 กิโลเมตร\nคำตอบ: 150 กิโลเมตร\n\nตอนนี้แก้โจทย์นี้:\nโจทย์: ร้านกาแฟขายกาแฟแก้วละ 50 บาท ต้นทุน 20 บาท ถ้าวันนี้ขายได้ 45 แก้ว \n       และมีค่าเช่าร้านวันละ 500 บาท กำไรสุทธิวันนี้เท่าไหร่?\nวิธีคิด:\n\"\"\"\n\nresponse = call_gemini(few_shot_cot_prompt)\nprint_section(\"Few-shot CoT Result\", response)\n\n#%% [markdown]\n# ### 📊 CoT vs Standard: เมื่อไหร่ควรใช้ CoT\n# \n# ```\n# ┌────────────────────────────────────────────────────────────┐\n# │            🎯 WHEN TO USE CoT?                             │\n# │                                                            │\n# │  ✅ USE CoT:                    ❌ DON'T NEED CoT:         │\n# │  • Math problems               • Simple factual Q&A       │\n# │  • Logic puzzles                • Translation              │\n# │  • Multi-step reasoning         • Summarization            │\n# │  • Cause-effect analysis        • Sentiment (simple)       │\n# │  • Decision making              • Fill-in-the-blank        │\n# │  • Code debugging               • Short creative writing   │\n# │  • Business analysis                                       │\n# │  • Scientific reasoning                                    │\n# │                                                            │\n# │  💡 Rule of thumb:                                         │\n# │  ถ้ามนุษย์ต้อง \"คิด\" หลายขั้นตอน → ใช้ CoT              │\n# │  ถ้ามนุษย์ตอบได้ทันที → ไม่จำเป็น                         │\n# └────────────────────────────────────────────────────────────┘\n# ```\n\n#%% [markdown]\n# ### 📝 แบบฝึกหัด 2: Chain-of-Thought\n# \n# **โจทย์**: ใช้ CoT แก้ปัญหานี้:\n# \"บริษัทมีพนักงาน 50 คน ต้องการเพิ่มเป็น 75 คนภายใน 6 เดือน \n# ถ้าทุก 2 เดือนรับพนักงานใหม่จำนวนเท่าๆ กัน แต่ละรอบต้องรับกี่คน?\n# และถ้าค่าใช้จ่ายในการรับพนักงาน 1 คน = 15,000 บาท ต้องใช้งบประมาณเท่าไหร่?\"\n\n#%%\n# === แบบฝึกหัด 2: เขียน Code ของคุณที่นี่ ===\n\n# TODO: สร้าง CoT prompt\ncot_exercise_prompt = \"\"\"\n# เขียน prompt ของคุณที่นี่\n\n\n\"\"\"\n\n# response = call_gemini(cot_exercise_prompt)\n# print(response)\n\n#%% [markdown]\n# ---\n# # 📘 Part 3: Self-Consistency Chain-of-Thought\n# **ระดับความยาก: ⭐⭐⭐ (กลาง-สูง)**\n# \n# ---\n# ## 📖 ทฤษฎีเชิงลึก\n# \n# **Self-Consistency CoT** คือเทคนิคที่สร้างคำตอบหลายๆ ครั้ง \n# แล้วเลือกคำตอบที่ consistent (สอดคล้องกัน) มากที่สุด\n# \n# ### 📊 Infographic: Self-Consistency ทำงานอย่างไร\n# \n# ```\n# ╔══════════════════════════════════════════════════════════════════════╗\n# ║              🔄 SELF-CONSISTENCY MECHANISM                          ║\n# ╠══════════════════════════════════════════════════════════════════════╣\n# ║                                                                      ║\n# ║                      ┌──────────┐                                    ║\n# ║                      │ Question │                                    ║\n# ║                      └─────┬────┘                                    ║\n# ║                ┌───────────┼───────────┐                             ║\n# ║                │           │           │                             ║\n# ║                ▼           ▼           ▼                             ║\n# ║         ┌──────────┐ ┌──────────┐ ┌──────────┐                      ║\n# ║         │ Path 1   │ │ Path 2   │ │ Path 3   │   (temperature > 0) ║\n# ║         │ Step A   │ │ Step X   │ │ Step P   │                      ║\n# ║         │ Step B   │ │ Step Y   │ │ Step Q   │                      ║\n# ║         │ Step C   │ │ Step Z   │ │ Step R   │                      ║\n# ║         └────┬─────┘ └────┬─────┘ └────┬─────┘                      ║\n# ║              │           │           │                               ║\n# ║              ▼           ▼           ▼                               ║\n# ║         Answer: 42   Answer: 42   Answer: 38                        ║\n# ║                                                                      ║\n# ║         ┌─────────────────────────────┐                              ║\n# ║         │     📊 Majority Vote        │                              ║\n# ║         │     42: ██████████ (2 votes)│                              ║\n# ║         │     38: █████ (1 vote)      │                              ║\n# ║         │                             │                              ║\n# ║         │     ✅ Final Answer: 42     │                              ║\n# ║         └─────────────────────────────┘                              ║\n# ║                                                                      ║\n# ║  📈 Wang et al. (2023): Self-Consistency improves CoT by 1-24%      ║\n# ║     on arithmetic and commonsense reasoning benchmarks              ║\n# ╚══════════════════════════════════════════════════════════════════════╝\n# ```\n# \n# ### 🔑 ข้อสำคัญ\n# \n# ```\n# ┌─────────────────────────────────────────────────────────┐\n# │            ⚙️ SELF-CONSISTENCY PARAMETERS               │\n# │                                                          │\n# │  1. Number of Samples (N):                               │\n# │     • N = 3-5: ดีสำหรับงานทั่วไป                         │\n# │     • N = 10-20: ดีสำหรับงานซับซ้อน                      │\n# │     • N = 40+: ตามงานวิจัย (แต่ cost สูง)               │\n# │                                                          │\n# │  2. Temperature:                                         │\n# │     • 0.5-0.8: ดีสำหรับ diverse but reasonable paths    │\n# │     • > 1.0: อาจได้ paths ที่ไม่ make sense             │\n# │                                                          │\n# │  3. Voting Strategy:                                     │\n# │     • Majority Vote: เลือกคำตอบที่มากที่สุด              │\n# │     • Weighted Vote: ให้น้ำหนักตาม confidence            │\n# │     • Plurality: เลือก unique answer ที่มากที่สุด        │\n# │                                                          │\n# │  4. Cost Consideration:                                  │\n# │     • N samples = N × API calls = N × cost              │\n# │     • Balance: quality vs. budget                        │\n# └─────────────────────────────────────────────────────────┘\n# ```\n\n#%%\n# === Lab 3.1: Self-Consistency Implementation ===\n\ndef self_consistency_cot(prompt: str, num_samples: int = 5, temperature: float = 0.7) -> Dict:\n    \"\"\"\n    สร้างคำตอบหลายครั้งและเลือกคำตอบที่ consistent ที่สุด\n    \"\"\"\n    responses = []\n    final_answers = []\n    \n    for i in range(num_samples):\n        response = call_gemini(prompt, temperature=temperature)\n        responses.append(response)\n        \n        # พยายามดึงคำตอบสุดท้าย\n        numbers = re.findall(r'\\b\\d+(?:,\\d{3})*(?:\\.\\d+)?\\b', response)\n        if numbers:\n            final_answers.append(numbers[-1].replace(',', ''))\n        \n        print(f\"📝 Sample {i+1}: {response[:200]}...\")\n        print(\"-\" * 50)\n        time.sleep(1)  # Rate limiting\n    \n    # นับความถี่ของคำตอบ\n    answer_counts = Counter(final_answers)\n    \n    return {\n        \"all_responses\": responses,\n        \"extracted_answers\": final_answers,\n        \"answer_distribution\": dict(answer_counts),\n        \"most_common\": answer_counts.most_common(1)[0] if answer_counts else None,\n        \"consistency_score\": max(answer_counts.values()) / len(final_answers) if final_answers else 0\n    }\n\n#%%\n# ทดสอบ Self-Consistency CoT\nproblem_prompt = \"\"\"\nแก้ปัญหานี้โดยแสดงวิธีคิดทีละขั้นตอน:\n\nร้านอาหารมีโต๊ะ 12 โต๊ะ แต่ละโต๊ะนั่งได้ 4 คน\nวันนี้มีลูกค้ามา 35 คน และอีก 2 กลุ่ม กลุ่มละ 6 คน จะมาทีหลัง\nถามว่า: ร้านสามารถรองรับลูกค้าทั้งหมดได้หรือไม่? \nและถ้าได้ จะเหลือที่นั่งว่างกี่ที่?\n\nแสดงวิธีคิด แล้วสรุปคำตอบเป็นตัวเลขที่นั่งที่เหลือ\n\"\"\"\n\nprint(\"🔄 Running Self-Consistency CoT (3 samples)...\")\nresult = self_consistency_cot(problem_prompt, num_samples=3)\n\nprint(\"\\n\" + \"=\"*50)\nprint(\"📊 Results Summary:\")\nprint(f\"  Answer Distribution: {result['answer_distribution']}\")\nprint(f\"  Most Common Answer: {result['most_common']}\")\nprint(f\"  Consistency Score: {result['consistency_score']:.1%}\")\n\n#%%\n# === Lab 3.2: Advanced Self-Consistency with Weighted Voting ===\n\ndef advanced_self_consistency(prompt: str, num_samples: int = 5) -> Dict:\n    \"\"\"\n    Self-Consistency พร้อม confidence scoring\n    \"\"\"\n    results = []\n    \n    enhanced_prompt = f\"\"\"\n{prompt}\n\nหลังจากคิดเสร็จ กรุณาให้:\n1. คำตอบสุดท้าย (ตัวเลขหรือ Yes/No)\n2. ความมั่นใจในคำตอบ (1-10)\n\nFormat:\nคำตอบ: [your answer]\nความมั่นใจ: [1-10]\n\"\"\"\n    \n    for i in range(num_samples):\n        response = call_gemini(enhanced_prompt, temperature=0.8)\n        \n        # Extract answer and confidence\n        answer_match = re.search(r'คำตอบ[:\\s]*([^\\n]+)', response)\n        confidence_match = re.search(r'ความมั่นใจ[:\\s]*(\\d+)', response)\n        \n        answer = answer_match.group(1).strip() if answer_match else \"N/A\"\n        confidence = int(confidence_match.group(1)) if confidence_match else 5\n        \n        results.append({\n            \"full_response\": response,\n            \"answer\": answer,\n            \"confidence\": confidence\n        })\n        \n        print(f\"  Sample {i+1}: Answer={answer}, Confidence={confidence}/10\")\n        time.sleep(1)\n    \n    # Weighted voting\n    weighted_answers = {}\n    for r in results:\n        ans = r[\"answer\"]\n        conf = r[\"confidence\"]\n        weighted_answers[ans] = weighted_answers.get(ans, 0) + conf\n    \n    best_answer = max(weighted_answers, key=weighted_answers.get)\n    \n    return {\n        \"results\": results,\n        \"weighted_scores\": weighted_answers,\n        \"best_answer\": best_answer,\n        \"total_confidence\": weighted_answers[best_answer]\n    }\n\n#%%\n# ทดสอบ Advanced Self-Consistency\ntest_prompt = \"\"\"\nบริษัทกำลังพิจารณาขยายสาขาใหม่ มีข้อมูล:\n- ต้นทุนเปิดสาขา: 2,000,000 บาท\n- รายได้คาดการณ์ต่อเดือน: 350,000 บาท\n- ค่าใช้จ่ายต่อเดือน: 280,000 บาท\n\nคำถาม: กี่เดือนจึงจะคืนทุน? (ตอบเป็นจำนวนเต็ม)\n\"\"\"\n\nprint(\"🔄 Running Advanced Self-Consistency...\")\nadvanced_result = advanced_self_consistency(test_prompt, num_samples=3)\n\nprint(\"\\n📊 Weighted Voting Results:\")\nprint(f\"  Weighted Scores: {advanced_result['weighted_scores']}\")\nprint(f\"  Best Answer: {advanced_result['best_answer']}\")\n\n#%% [markdown]\n# ---\n# # 📘 Part 4: Prompt Chaining\n# **ระดับความยาก: ⭐⭐⭐ (กลาง-สูง)**\n# \n# ---\n# ## 📖 ทฤษฎีเชิงลึก\n# \n# **Prompt Chaining** คือเทคนิคที่แบ่งงานซับซ้อนออกเป็นขั้นตอนย่อยๆ \n# โดย output ของ prompt หนึ่งจะเป็น input ของ prompt ถัดไป\n# \n# ### 📊 Infographic: Prompt Chaining Architecture\n# \n# ```\n# ╔══════════════════════════════════════════════════════════════════════╗\n# ║              🔗 PROMPT CHAINING ARCHITECTURE                        ║\n# ╠══════════════════════════════════════════════════════════════════════╣\n# ║                                                                      ║\n# ║  Single Monolithic Prompt (❌ ไม่แนะนำสำหรับงานซับซ้อน):           ║\n# ║  ┌──────────────────────────────────────────────────┐               ║\n# ║  │  \"ทำ A, B, C, D, E ทั้งหมดพร้อมกัน\"              │               ║\n# ║  │  → มักได้ผลลัพธ์ไม่ดี เพราะ context ยาวเกินไป    │               ║\n# ║  └──────────────────────────────────────────────────┘               ║\n# ║                                                                      ║\n# ║  Prompt Chaining (✅ แนะนำ):                                        ║\n# ║  ┌─────┐    ┌─────┐    ┌─────┐    ┌─────┐    ┌─────────┐          ║\n# ║  │ P1  │───▶│ P2  │───▶│ P3  │───▶│ P4  │───▶│ Final   │          ║\n# ║  │Extract│   │Analyze│  │Plan │   │Refine│   │ Output  │          ║\n# ║  └─────┘    └─────┘    └─────┘    └─────┘    └─────────┘          ║\n# ║     │          │          │          │                              ║\n# ║     └──context──┘──context──┘──context──┘                           ║\n# ║                                                                      ║\n# ║  ✨ Benefits:                                                        ║\n# ║  • แต่ละขั้นตอนมี focus ชัดเจน                                      ║\n# ║  • ควบคุมคุณภาพแต่ละขั้นตอนได้                                     ║\n# ║  • Debug ง่าย (รู้ว่าผิดตรงไหน)                                    ║\n# ║  • สามารถใช้ model ต่างกันในแต่ละขั้น                              ║\n# ║  • เพิ่ม/ลด ขั้นตอนได้ง่าย                                         ║\n# ║                                                                      ║\n# ║  ⚠️ Limitations:                                                     ║\n# ║  • Latency สูงขึ้น (หลาย API calls)                                ║\n# ║  • Error propagation (ผิดขั้นแรก → ผิดหมด)                        ║\n# ║  • Cost เพิ่มขึ้นตามจำนวนขั้นตอน                                   ║\n# ╚══════════════════════════════════════════════════════════════════════╝\n# ```\n# \n# ### 📐 Chaining Patterns ที่พบบ่อย\n# \n# ```\n# Pattern 1: Sequential (ลำดับ)\n# ┌─────┐ → ┌─────┐ → ┌─────┐ → ┌─────┐\n# │  A  │   │  B  │   │  C  │   │  D  │\n# └─────┘   └─────┘   └─────┘   └─────┘\n# ตัวอย่าง: Extract → Analyze → Summarize → Format\n#\n# Pattern 2: Fan-out (กระจาย)\n#                    ┌─────┐\n#               ┌──▶│  B1 │\n# ┌─────┐      │    └─────┘\n# │  A  │──────┤    ┌─────┐\n# └─────┘      ├──▶│  B2 │\n#              │    └─────┘\n#              │    ┌─────┐\n#              └──▶│  B3 │\n#                   └─────┘\n# ตัวอย่าง: แบ่งบทความ → วิเคราะห์แต่ละส่วนแยก\n#\n# Pattern 3: Fan-in (รวม)\n# ┌─────┐\n# │  A  │────┐\n# └─────┘    │    ┌─────┐\n# ┌─────┐    ├──▶│  D  │\n# │  B  │────┤    └─────┘\n# └─────┘    │\n# ┌─────┐    │\n# │  C  │────┘\n# └─────┘\n# ตัวอย่าง: SWOT + Financial + Market → รวมเป็น Business Plan\n#\n# Pattern 4: Conditional (เงื่อนไข)\n# ┌─────┐      YES → ┌─────┐\n# │  A  │──────────▶│  B  │\n# └─────┘      NO  → ┌─────┐\n#                     │  C  │\n#                     └─────┘\n# ตัวอย่าง: ตรวจสอบ input → ถ้า valid → process, ถ้า invalid → fix\n# ```\n\n#%%\n# === Lab 4.1: Basic Prompt Chaining ===\n# Task: วิเคราะห์และสรุปบทความ\n\ndef prompt_chain_article_analysis(article: str) -> Dict:\n    \"\"\"\n    Chain สำหรับวิเคราะห์บทความแบบหลายขั้นตอน\n    \"\"\"\n    results = {}\n    \n    # Step 1: สกัดข้อมูลสำคัญ\n    step1_prompt = f\"\"\"\nวิเคราะห์บทความต่อไปนี้และสกัดข้อมูลสำคัญ:\n\nบทความ:\n{article}\n\nกรุณาระบุ:\n1. หัวข้อหลัก (Main Topic)\n2. ประเด็นสำคัญ (Key Points) - ระบุ 3-5 ประเด็น\n3. ข้อมูลตัวเลข/สถิติ (ถ้ามี)\n4. บุคคล/องค์กรที่เกี่ยวข้อง\n\nตอบในรูปแบบ structured format\n\"\"\"\n    \n    response1 = call_gemini(step1_prompt)\n    results[\"step1_extraction\"] = response1\n    print(\"✅ Step 1: Information Extraction Complete\")\n    print(response1)\n    print(\"-\" * 50)\n    \n    # Step 2: วิเคราะห์ sentiment และ tone\n    step2_prompt = f\"\"\"\nจากข้อมูลที่สกัดได้:\n{results['step1_extraction']}\n\nกรุณาวิเคราะห์:\n1. Sentiment ของบทความ (บวก/ลบ/กลาง)\n2. Tone การเขียน (เป็นทางการ/ไม่เป็นทางการ/วิชาการ)\n3. กลุ่มเป้าหมายของบทความ\n4. วัตถุประสงค์ของผู้เขียน\n\"\"\"\n    \n    response2 = call_gemini(step2_prompt)\n    results[\"step2_analysis\"] = response2\n    print(\"✅ Step 2: Sentiment & Tone Analysis Complete\")\n    print(response2)\n    print(\"-\" * 50)\n    \n    # Step 3: สร้างสรุปและข้อเสนอแนะ\n    step3_prompt = f\"\"\"\nจากการวิเคราะห์:\n\nข้อมูลสำคัญ:\n{results['step1_extraction']}\n\nการวิเคราะห์:\n{results['step2_analysis']}\n\nกรุณาสร้าง:\n1. สรุปบทความ (3-5 ประโยค)\n2. ข้อเสนอแนะสำหรับผู้อ่าน\n3. คำถามที่ควรพิจารณาต่อ\n4. แหล่งข้อมูลเพิ่มเติมที่ควรศึกษา (แนะนำ)\n\"\"\"\n    \n    response3 = call_gemini(step3_prompt)\n    results[\"step3_summary\"] = response3\n    print(\"✅ Step 3: Summary & Recommendations Complete\")\n    print(response3)\n    \n    return results\n\n#%%\n# ทดสอบ Prompt Chaining\nsample_article = \"\"\"\nกรุงเทพฯ - ธนาคารแห่งประเทศไทยประกาศคงอัตราดอกเบี้ยนโยบายไว้ที่ 2.50% \nในการประชุมคณะกรรมการนโยบายการเงิน (กนง.) เมื่อวานนี้ โดยมีมติเอกฉันท์ 7-0 เสียง\n\nนายเศรษฐพุฒิ สุทธิวาทนฤพุฒิ ผู้ว่าการธนาคารแห่งประเทศไทย กล่าวว่า \nการคงอัตราดอกเบี้ยครั้งนี้เพื่อประเมินผลกระทบของนโยบายการเงินที่ผ่านมา \nและติดตามสถานการณ์เศรษฐกิจโลกที่ยังมีความไม่แน่นอน\n\nทั้งนี้ GDP ไทยไตรมาสล่าสุดขยายตัว 2.8% สูงกว่าคาดการณ์ที่ 2.5% \nขณะที่เงินเฟ้อทั่วไปอยู่ที่ 0.8% ซึ่งอยู่ในกรอบเป้าหมาย 1-3%\n\nนักวิเคราะห์คาดว่า กนง. จะคงดอกเบี้ยไปจนถึงกลางปีหน้า \nก่อนพิจารณาปรับลดหากเศรษฐกิจชะลอตัว\n\"\"\"\n\nprint(\"🔗 Starting Prompt Chain Analysis...\")\nchain_results = prompt_chain_article_analysis(sample_article)\n\n#%%\n# === Lab 4.2: Complex Prompt Chaining - Business Analysis ===\n\ndef business_analysis_chain(company_data: str) -> Dict:\n    \"\"\"\n    Chain สำหรับวิเคราะห์ธุรกิจแบบครบวงจร\n    \"\"\"\n    results = {}\n    \n    # Chain 1: SWOT Analysis\n    chain1 = f\"\"\"\nวิเคราะห์ SWOT จากข้อมูลบริษัท:\n{company_data}\n\nระบุ:\n- Strengths (จุดแข็ง): 3-4 ข้อ\n- Weaknesses (จุดอ่อน): 3-4 ข้อ\n- Opportunities (โอกาส): 3-4 ข้อ\n- Threats (ภัยคุกคาม): 3-4 ข้อ\n\"\"\"\n    \n    response1 = call_gemini(chain1)\n    results[\"swot\"] = response1\n    print(\"✅ Chain 1: SWOT Analysis Done\")\n    \n    # Chain 2: Strategic Recommendations based on SWOT\n    chain2 = f\"\"\"\nจาก SWOT Analysis:\n{results['swot']}\n\nสร้างกลยุทธ์:\n1. SO Strategy (ใช้จุดแข็งคว้าโอกาส)\n2. WO Strategy (แก้จุดอ่อนด้วยโอกาส)\n3. ST Strategy (ใช้จุดแข็งรับมือภัยคุกคาม)\n4. WT Strategy (ลดจุดอ่อนและหลีกเลี่ยงภัยคุกคาม)\n\nระบุ action items ที่ชัดเจนสำหรับแต่ละกลยุทธ์\n\"\"\"\n    \n    response2 = call_gemini(chain2)\n    results[\"strategies\"] = response2\n    print(\"✅ Chain 2: Strategic Recommendations Done\")\n    \n    # Chain 3: Implementation Roadmap\n    chain3 = f\"\"\"\nจากกลยุทธ์ที่วางไว้:\n{results['strategies']}\n\nสร้าง Implementation Roadmap:\n1. Quick Wins (0-3 เดือน): สิ่งที่ทำได้ทันที\n2. Short-term (3-6 เดือน): โปรเจกต์ระยะสั้น\n3. Medium-term (6-12 เดือน): แผนระยะกลาง\n4. Long-term (1-2 ปี): วิสัยทัศน์ระยะยาว\n\nระบุ KPIs สำหรับแต่ละช่วง\n\"\"\"\n    \n    response3 = call_gemini(chain3)\n    results[\"roadmap\"] = response3\n    print(\"✅ Chain 3: Implementation Roadmap Done\")\n    \n    return results\n\n#%%\n# ทดสอบ Business Analysis Chain\ncompany_info = \"\"\"\nบริษัท TechStart Co., Ltd.\n- ธุรกิจ: พัฒนาแอปพลิเคชันมือถือ\n- พนักงาน: 25 คน\n- รายได้ปีที่แล้ว: 15 ล้านบาท\n- ลูกค้าหลัก: SMEs ในประเทศไทย\n- จุดเด่น: ทีมพัฒนาฝีมือดี, ราคาแข่งขันได้\n- ปัญหา: การตลาดยังไม่แข็งแรง, ขาด Product Manager\n- ตลาด: การแข่งขันสูงขึ้น, มี Freelancer เพิ่มขึ้น\n- โอกาส: Digital Transformation กำลังโต, รัฐบาลส่งเสริม Tech Startup\n\"\"\"\n\nprint(\"🏢 Starting Business Analysis Chain...\")\nbusiness_results = business_analysis_chain(company_info)\n\n#%% [markdown]\n# ---\n# # 📘 Part 5: Reflection\n# **ระดับความยาก: ⭐⭐⭐⭐ (สูง)**\n# \n# ---\n# ## 📖 ทฤษฎีเชิงลึก\n# \n# **Reflection** คือเทคนิคที่ให้ LLM ทบทวนและประเมินคำตอบของตัวเอง\n# แล้วปรับปรุงให้ดีขึ้น คล้ายกับการ \"อ่านทวน\" ก่อนส่งงาน\n# \n# ### 📊 Infographic: Reflection Cycle\n# \n# ```\n# ╔══════════════════════════════════════════════════════════════════╗\n# ║                 🔍 REFLECTION CYCLE                              ║\n# ╠══════════════════════════════════════════════════════════════════╣\n# ║                                                                  ║\n# ║         ┌───────────────────┐                                    ║\n# ║         │  1. GENERATE      │                                    ║\n# ║         │  สร้างคำตอบแรก    │                                    ║\n# ║         └────────┬──────────┘                                    ║\n# ║                  │                                               ║\n# ║                  ▼                                               ║\n# ║         ┌───────────────────┐                                    ║\n# ║         │  2. REFLECT       │                                    ║\n# ║         │  วิเคราะห์จุดอ่อน  │◄──────────┐                       ║\n# ║         └────────┬──────────┘           │                        ║\n# ║                  │                      │                        ║\n# ║                  ▼                      │  Loop until            ║\n# ║         ┌───────────────────┐           │  satisfied             ║\n# ║         │  3. IMPROVE       │           │  (or max               ║\n# ║         │  ปรับปรุงตาม      │───────────┘   iterations)          ║\n# ║         │  ผลวิเคราะห์      │                                    ║\n# ║         └────────┬──────────┘                                    ║\n# ║                  │                                               ║\n# ║                  ▼                                               ║\n# ║         ┌───────────────────┐                                    ║\n# ║         │  4. FINAL OUTPUT  │                                    ║\n# ║         │  คำตอบที่ปรับปรุง  │                                    ║\n# ║         └───────────────────┘                                    ║\n# ║                                                                  ║\n# ║  Reflection vs. เทคนิคอื่น:                                     ║\n# ║  ┌──────────────┬────────────────────────────────────┐          ║\n# ║  │ Technique    │ Focus                               │          ║\n# ║  │──────────────│────────────────────────────────────│          ║\n# ║  │ Reflection   │ ทบทวนภาพรวม, ปรับปรุงทั่วไป        │          ║\n# ║  │ Self-Feedback│ ประเมินตามเกณฑ์เฉพาะ               │          ║\n# ║  │ Self-Critique│ วิพากษ์อย่างเข้มงวด, หาข้อผิดพลาด  │          ║\n# ║  └──────────────┴────────────────────────────────────┘          ║\n# ╚══════════════════════════════════════════════════════════════════╝\n# ```\n# \n# ### 🔬 งานวิจัยที่เกี่ยวข้อง\n# - **Madaan et al. (2023)** — \"Self-Refine\" framework\n#   LLM ที่ refine output ของตัวเองซ้ำๆ ได้ผลดีกว่า single-pass 5-20%\n# - **Shinn et al. (2023)** — \"Reflexion: Language Agents with Verbal Reinforcement Learning\"\n#   ใช้ verbal reflection เป็น feedback signal แทน scalar reward\n\n#%%\n# === Lab 5.1: Basic Reflection Pattern ===\n\ndef reflection_prompt(task: str) -> Dict:\n    \"\"\"\n    Reflection pattern: Generate -> Reflect -> Improve\n    \"\"\"\n    results = {}\n    \n    # Step 1: Initial Response\n    initial_prompt = f\"\"\"\n{task}\n\nให้คำตอบที่ดีที่สุดของคุณ\n\"\"\"\n    \n    response1 = call_gemini(initial_prompt)\n    results[\"initial_response\"] = response1\n    print_section(\"Initial Response\", response1, \"📝\")\n    \n    # Step 2: Reflection\n    reflect_prompt = f\"\"\"\nทบทวนคำตอบต่อไปนี้:\n\nคำถาม/งาน:\n{task}\n\nคำตอบที่ให้ไป:\n{results['initial_response']}\n\nกรุณาวิเคราะห์:\n1. จุดแข็งของคำตอบ (2-3 ข้อ)\n2. จุดอ่อนหรือสิ่งที่ขาดหาย (2-3 ข้อ)\n3. ข้อผิดพลาดที่อาจมี\n4. สิ่งที่ควรเพิ่มเติม\n\"\"\"\n    \n    response2 = call_gemini(reflect_prompt)\n    results[\"reflection\"] = response2\n    print_section(\"Reflection\", response2, \"🔍\")\n    \n    # Step 3: Improved Response\n    improvement_prompt = f\"\"\"\nจากการทบทวน:\n\nคำถาม/งานเดิม:\n{task}\n\nคำตอบเดิม:\n{results['initial_response']}\n\nผลการวิเคราะห์:\n{results['reflection']}\n\nกรุณาสร้างคำตอบใหม่ที่ปรับปรุงแล้ว โดยแก้ไขจุดอ่อนและเพิ่มเติมสิ่งที่ขาดหาย\n\"\"\"\n    \n    response3 = call_gemini(improvement_prompt)\n    results[\"improved_response\"] = response3\n    print_section(\"Improved Response\", response3, \"✨\")\n    \n    return results\n\n#%%\n# ทดสอบ Reflection\ntask = \"\"\"\nเขียนอีเมลขอโทษลูกค้าที่สินค้าส่งล่าช้า 3 วัน \nโดยให้เหตุผลและเสนอชดเชย\n\"\"\"\n\nprint(\"🔄 Starting Reflection Process...\")\nreflection_results = reflection_prompt(task)\n\n#%%\n# === Lab 5.2: Multi-round Reflection ===\n\ndef iterative_reflection(task: str, max_iterations: int = 3) -> Dict:\n    \"\"\"\n    Reflection หลายรอบจนกว่าจะพอใจ\n    \"\"\"\n    results = {\"iterations\": []}\n    \n    current_response = None\n    \n    for i in range(max_iterations):\n        print(f\"\\n🔄 Iteration {i + 1}\")\n        print(\"=\" * 50)\n        \n        if current_response is None:\n            # First iteration: Generate initial response\n            prompt = f\"{task}\\n\\nให้คำตอบที่ดีที่สุดของคุณ\"\n        else:\n            # Subsequent iterations: Reflect and improve\n            prompt = f\"\"\"\nคำถาม/งาน:\n{task}\n\nคำตอบปัจจุบัน:\n{current_response}\n\nกรุณา:\n1. วิเคราะห์ว่าคำตอบนี้ดีพอหรือยัง (ตอบ YES หรือ NO)\n2. ถ้า NO: ระบุจุดที่ต้องปรับปรุงและสร้างคำตอบใหม่ที่ดีกว่า\n3. ถ้า YES: ยืนยันคำตอบและอธิบายว่าทำไมถึงดีแล้ว\n\nFormat:\nSTATUS: [YES/NO]\nANALYSIS: [การวิเคราะห์]\nRESPONSE: [คำตอบ - ถ้า NO ให้คำตอบใหม่, ถ้า YES ให้คำตอบเดิม]\n\"\"\"\n        \n        response = call_gemini(prompt, temperature=0.3)\n        \n        results[\"iterations\"].append({\n            \"iteration\": i + 1,\n            \"response\": response\n        })\n        \n        print(response[:500] + \"...\" if len(response) > 500 else response)\n        \n        # Check if satisfied\n        if \"STATUS: YES\" in response.upper() or \"STATUS:YES\" in response.upper():\n            print(\"\\n✅ Reflection Complete - Satisfied with response\")\n            break\n        \n        current_response = response\n        time.sleep(1)\n    \n    results[\"final_response\"] = results[\"iterations\"][-1][\"response\"]\n    results[\"num_iterations\"] = len(results[\"iterations\"])\n    return results\n\n#%%\n# ทดสอบ Iterative Reflection\ncomplex_task = \"\"\"\nออกแบบ Loyalty Program สำหรับร้านกาแฟที่มี 3 สาขา\nโดยต้องคำนึงถึง:\n- งบประมาณจำกัด\n- ต้องการเพิ่ม repeat customers 30%\n- ใช้เทคโนโลยีที่เรียบง่าย\n\"\"\"\n\nprint(\"🔄 Starting Iterative Reflection...\")\niter_results = iterative_reflection(complex_task, max_iterations=3)\nprint(f\"\\n📊 Total iterations: {iter_results['num_iterations']}\")\n\n#%% [markdown]\n# ---\n# # 📘 Part 6: Tree-of-Thought (ToT)\n# **ระดับความยาก: ⭐⭐⭐⭐⭐ (สูงมาก)**\n# \n# ---\n# ## 📖 ทฤษฎีเชิงลึก\n# \n# **Tree-of-Thought (ToT)** คือเทคนิคที่สำรวจหลายเส้นทางการคิดพร้อมกัน\n# เหมือนการเล่นหมากรุกที่พิจารณาหลายๆ ตาล่วงหน้า\n# \n# ### 📊 Infographic: ToT Architecture\n# \n# ```\n# ╔══════════════════════════════════════════════════════════════════════╗\n# ║                 🌳 TREE-OF-THOUGHT ARCHITECTURE                     ║\n# ╠══════════════════════════════════════════════════════════════════════╣\n# ║                                                                      ║\n# ║  CoT (Linear):     ToT (Branching):                                 ║\n# ║  ●→●→●→●           ●─┬─●─┬─●──── ✅ Best                            ║\n# ║  (1 path)          │  │  └─●──── ❌ Pruned                          ║\n# ║                    │  └─●─┬─●──── ❌ Pruned                          ║\n# ║                    │     └─●──── ❌ Pruned                           ║\n# ║                    └─●─┬─●──── ❌ Pruned                             ║\n# ║                       └─●──── ❌ Pruned                              ║\n# ║                                                                      ║\n# ║  Process:                                                            ║\n# ║  ┌──────────┐  ┌──────────┐  ┌──────────┐  ┌──────────┐            ║\n# ║  │1.GENERATE│─▶│2.EVALUATE│─▶│3.EXPAND  │─▶│4.SELECT  │            ║\n# ║  │ Branches │  │ Score    │  │ Best     │  │ Final    │            ║\n# ║  │ (BFS/DFS)│  │ each    │  │ branch   │  │ answer   │            ║\n# ║  └──────────┘  └──────────┘  └──────────┘  └──────────┘            ║\n# ║                                                                      ║\n# ║  🔑 Search Strategies:                                               ║\n# ║  ┌───────────────────────────────────────────────────────┐          ║\n# ║  │ BFS (Breadth-First)     │ DFS (Depth-First)          │          ║\n# ║  │ • ดูทุก branch ในระดับ   │ • ไล่ลึกทีละ branch        │          ║\n# ║  │   เดียวกันก่อน           │ • ใช้ memory น้อยกว่า      │          ║\n# ║  │ • เหมาะกับ decision     │ • เหมาะกับปัญหาที่มี      │          ║\n# ║  │   making tasks          │   solution ลึก             │          ║\n# ║  └───────────────────────────────────────────────────────┘          ║\n# ║                                                                      ║\n# ║  📈 Yao et al. (2023): ToT improves GPT-4 on \"Game of 24\"          ║\n# ║     from 4% (CoT) to 74% success rate                               ║\n# ╚══════════════════════════════════════════════════════════════════════╝\n# ```\n# \n# ### ToT vs CoT vs Self-Consistency\n# ```\n# ┌─────────────────────────────────────────────────────────────┐\n# │              🔄 TECHNIQUE COMPARISON                         │\n# │                                                               │\n# │ Technique          │ Paths │ Evaluation │ Best For           │\n# │────────────────────│───────│────────────│────────────────────│\n# │ CoT                │   1   │    No      │ Simple reasoning   │\n# │ Self-Consistency   │   N   │  Voting    │ Math, factual Q&A  │\n# │ Tree-of-Thought    │   N   │ Per-step   │ Complex planning,  │\n# │                    │       │ scoring    │ creative problem   │\n# │                    │       │            │ solving            │\n# └─────────────────────────────────────────────────────────────┘\n# ```\n\n#%%\n# === Lab 6.1: Tree-of-Thought Implementation ===\n\ndef tree_of_thought(problem: str, num_branches: int = 3, depth: int = 2) -> Dict:\n    \"\"\"\n    Tree-of-Thought: สำรวจหลายเส้นทางการแก้ปัญหา\n    \"\"\"\n    results = {\n        \"problem\": problem,\n        \"tree\": {}\n    }\n    \n    # Step 1: Generate initial branches (approaches)\n    generate_prompt = f\"\"\"\nพิจารณาปัญหา:\n{problem}\n\nสร้าง {num_branches} แนวทางการแก้ปัญหาที่แตกต่างกัน\nสำหรับแต่ละแนวทาง ให้:\n1. ชื่อแนวทาง\n2. แนวคิดหลัก\n3. ขั้นตอนแรกที่จะทำ\n\nFormat:\nAPPROACH 1:\nName: [ชื่อ]\nConcept: [แนวคิด]\nFirst Step: [ขั้นตอนแรก]\n\nAPPROACH 2:\n...\n\"\"\"\n    \n    response1 = call_gemini(generate_prompt, temperature=0.7)\n    results[\"tree\"][\"initial_branches\"] = response1\n    print_section(\"Initial Branches\", response1, \"🌳\")\n    \n    # Step 2: Evaluate each branch\n    evaluate_prompt = f\"\"\"\nประเมินแนวทางการแก้ปัญหาเหล่านี้:\n\nปัญหา:\n{problem}\n\nแนวทางที่เสนอ:\n{results['tree']['initial_branches']}\n\nสำหรับแต่ละแนวทาง ให้คะแนน 1-10 ในด้าน:\n1. ความเป็นไปได้ (Feasibility)\n2. ประสิทธิภาพ (Effectiveness)\n3. ความเสี่ยง (Risk - คะแนนสูง = ความเสี่ยงต่ำ)\n4. ทรัพยากรที่ต้องใช้ (Resources - คะแนนสูง = ใช้น้อย)\n\nสรุปคะแนนรวมและจัดอันดับ\n\"\"\"\n    \n    response2 = call_gemini(evaluate_prompt)\n    results[\"tree\"][\"evaluation\"] = response2\n    print_section(\"Branch Evaluation\", response2, \"📊\")\n    \n    # Step 3: Expand best branch\n    expand_prompt = f\"\"\"\nจากการประเมิน:\n{results['tree']['evaluation']}\n\nเลือกแนวทางที่ดีที่สุด และขยายรายละเอียด:\n\n1. อธิบายแผนการดำเนินงานโดยละเอียด\n2. ระบุ milestones สำคัญ\n3. คาดการณ์อุปสรรคและวิธีรับมือ\n4. ทรัพยากรที่ต้องเตรียม\n5. Timeline คร่าวๆ\n\"\"\"\n    \n    response3 = call_gemini(expand_prompt)\n    results[\"tree\"][\"expanded_solution\"] = response3\n    print_section(\"Expanded Best Solution\", response3, \"🎯\")\n    \n    return results\n\n#%%\n# ทดสอบ Tree-of-Thought\nbusiness_problem = \"\"\"\nร้านหนังสือเล็กๆ กำลังประสบปัญหายอดขายลดลง 40% \nเนื่องจากการแข่งขันจาก e-commerce และ e-book\nมีงบประมาณสำหรับปรับปรุง 500,000 บาท\nต้องการหาทางรอดภายใน 1 ปี\n\"\"\"\n\nprint(\"🌳 Starting Tree-of-Thought Analysis...\")\ntot_results = tree_of_thought(business_problem)\n\n#%% [markdown]\n# ---\n# # 📘 Part 7: Self-Feedback\n# **ระดับความยาก: ⭐⭐⭐⭐ (สูง)**\n# \n# ---\n# ## 📖 ทฤษฎีเชิงลึก\n# \n# **Self-Feedback** คือเทคนิคที่ให้ LLM สร้าง feedback ให้ตัวเองตามเกณฑ์ที่กำหนด\n# แล้วใช้ feedback นั้นปรับปรุงคำตอบ\n# \n# ### ความแตกต่างจาก Reflection:\n# ```\n# ┌────────────────────────────────────────────────────────┐\n# │           🔍 REFLECTION vs SELF-FEEDBACK                │\n# │                                                          │\n# │  Reflection:                                             │\n# │  • ทบทวนภาพรวมทั่วไป                                    │\n# │  • ไม่มีเกณฑ์ตายตัว                                     │\n# │  • เหมือน \"อ่านทวน\"                                     │\n# │                                                          │\n# │  Self-Feedback:                                          │\n# │  • ใช้ rubric/criteria เฉพาะ                             │\n# │  • ให้คะแนนตามเกณฑ์                                     │\n# │  • เหมือน \"ตรวจตาม marking scheme\"                      │\n# │                                                          │\n# │  Example Criteria:                                       │\n# │  ┌──────────────────────────┐                           │\n# │  │ Accuracy     ████████░░ 8/10  │                      │\n# │  │ Completeness █████████░ 9/10  │                      │\n# │  │ Clarity      ███████░░░ 7/10  │  ← ต้องปรับปรุง    │\n# │  │ Engagement   ██████░░░░ 6/10  │  ← ต้องปรับปรุง    │\n# │  │ Objectivity  █████████░ 9/10  │                      │\n# │  └──────────────────────────┘                           │\n# └────────────────────────────────────────────────────────┘\n# ```\n\n#%%\n# === Lab 7.1: Self-Feedback with Criteria ===\n\ndef self_feedback_with_criteria(task: str, criteria: List[str]) -> Dict:\n    \"\"\"\n    Self-Feedback ด้วยเกณฑ์ที่กำหนด\n    \"\"\"\n    results = {}\n    \n    # Step 1: Generate initial response\n    response1 = call_gemini(task)\n    results[\"initial_response\"] = response1\n    print_section(\"Initial Response\", response1, \"📝\")\n    \n    # Step 2: Self-Feedback based on criteria\n    criteria_text = \"\\n\".join([f\"- {c}\" for c in criteria])\n    feedback_prompt = f\"\"\"\nประเมินคำตอบต่อไปนี้ตามเกณฑ์ที่กำหนด:\n\nงาน:\n{task}\n\nคำตอบ:\n{results['initial_response']}\n\nเกณฑ์การประเมิน:\n{criteria_text}\n\nสำหรับแต่ละเกณฑ์:\n1. ให้คะแนน 1-10\n2. อธิบายเหตุผล\n3. เสนอการปรับปรุง\n\nFormat:\n[เกณฑ์ 1]: [คะแนน]/10\n- เหตุผล: ...\n- การปรับปรุง: ...\n\"\"\"\n    \n    response2 = call_gemini(feedback_prompt)\n    results[\"self_feedback\"] = response2\n    print_section(\"Self-Feedback\", response2, \"🔍\")\n    \n    # Step 3: Improve based on feedback\n    improve_prompt = f\"\"\"\nปรับปรุงคำตอบตาม feedback:\n\nงานเดิม:\n{task}\n\nคำตอบเดิม:\n{results['initial_response']}\n\nFeedback:\n{results['self_feedback']}\n\nสร้างคำตอบใหม่ที่ปรับปรุงตาม feedback ทุกข้อ\n\"\"\"\n    \n    response3 = call_gemini(improve_prompt)\n    results[\"improved_response\"] = response3\n    print_section(\"Improved Response\", response3, \"✨\")\n    \n    return results\n\n#%%\n# ทดสอบ Self-Feedback\nwriting_task = \"\"\"\nเขียนบทความสั้น (200-300 คำ) เกี่ยวกับ \"ผลกระทบของ AI ต่อตลาดแรงงานไทย\"\n\"\"\"\n\ncriteria = [\n    \"ความถูกต้องของข้อมูล (Accuracy)\",\n    \"ความครบถ้วนของเนื้อหา (Completeness)\",\n    \"ความชัดเจนในการสื่อสาร (Clarity)\",\n    \"ความน่าสนใจ (Engagement)\",\n    \"ความเป็นกลาง (Objectivity)\"\n]\n\nprint(\"🔄 Starting Self-Feedback Process...\")\nsf_results = self_feedback_with_criteria(writing_task, criteria)\n\n#%% [markdown]\n# ---\n# # 📘 Part 8: Self-Critique\n# **ระดับความยาก: ⭐⭐⭐⭐⭐ (สูงมาก)**\n# \n# ---\n# ## 📖 ทฤษฎีเชิงลึก\n# \n# **Self-Critique** เป็นเทคนิคที่ LLM วิพากษ์คำตอบของตัวเองอย่างเข้มงวด\n# โดยพยายามหาข้อผิดพลาด จุดอ่อน และสมมติฐานที่ผิดพลาด\n# \n# ### 📊 Infographic: Self-Critique Intensity Spectrum\n# \n# ```\n# ╔══════════════════════════════════════════════════════════════════╗\n# ║            ⚔️ CRITIQUE INTENSITY SPECTRUM                       ║\n# ╠══════════════════════════════════════════════════════════════════╣\n# ║                                                                  ║\n# ║  Gentle                                           Adversarial   ║\n# ║  ←───────────────────────────────────────────────────────────→  ║\n# ║                                                                  ║\n# ║  Reflection    Self-Feedback    Self-Critique    Adversarial    ║\n# ║  \"ดีแล้ว       \"ตรงนี้ได้ 7/10  \"ข้อนี้ผิดเพราะ  \"พิสูจน์ว่า  ║\n# ║   แต่เพิ่ม     ควรปรับปรุง       สมมติฐานไม่มี    คำตอบนี้ผิด  ║\n# ║   อีกนิด\"      เรื่อง...\"       หลักฐานรองรับ\"   ทุกประเด็น\"  ║\n# ║                                                                  ║\n# ║  ✅ Best Uses:                                                   ║\n# ║  • Controversial topics     → ใช้ Adversarial Critique          ║\n# ║  • Factual writing          → ใช้ Self-Critique                 ║\n# ║  • Creative writing         → ใช้ Self-Feedback                 ║\n# ║  • Quick iteration          → ใช้ Reflection                    ║\n# ║                                                                  ║\n# ║  Checklist สิ่งที่ Self-Critique ตรวจ:                           ║\n# ║  □ Logical Fallacies (ตรรกะผิดพลาด)                             ║\n# ║  □ Unsupported Assumptions (สมมติฐานไร้หลักฐาน)                 ║\n# ║  □ Factual Errors (ข้อมูลผิด)                                   ║\n# ║  □ Weak Arguments (ข้อโต้แย้งอ่อน)                              ║\n# ║  □ Missing Perspectives (มุมมองที่ขาด)                          ║\n# ║  □ Cognitive Biases (อคติทางความคิด)                            ║\n# ║  □ Over-generalizations (เหมารวมเกินไป)                        ║\n# ║  □ Cherry-picking evidence (เลือกหลักฐานเฉพาะที่สนับสนุน)     ║\n# ╚══════════════════════════════════════════════════════════════════╝\n# ```\n\n#%%\n# === Lab 8.1: Adversarial Self-Critique ===\n\ndef adversarial_self_critique(task: str, num_rounds: int = 2) -> Dict:\n    \"\"\"\n    Self-Critique แบบ Adversarial: พยายามหักล้างคำตอบตัวเอง\n    \"\"\"\n    results = {\"rounds\": []}\n    current_response = None\n    current_critique = None\n    \n    for round_num in range(num_rounds):\n        print(f\"\\n🔄 Round {round_num + 1}\")\n        print(\"=\" * 50)\n        \n        # Generate/Improve response\n        if current_response is None:\n            gen_prompt = f\"{task}\\n\\nให้คำตอบที่ดีที่สุดของคุณ\"\n        else:\n            gen_prompt = f\"\"\"\nจากการวิพากษ์:\n{current_critique}\n\nงานเดิม:\n{task}\n\nคำตอบก่อนหน้า:\n{current_response}\n\nสร้างคำตอบใหม่ที่แก้ไขปัญหาทั้งหมดที่วิพากษ์ไว้\n\"\"\"\n        \n        response = call_gemini(gen_prompt)\n        current_response = response\n        print_section(\"Response\", current_response[:500], \"📝\")\n        \n        # Adversarial Critique\n        critique_prompt = f\"\"\"\nในฐานะนักวิจารณ์ที่เข้มงวด (Harsh Critic) วิพากษ์คำตอบนี้:\n\nงาน:\n{task}\n\nคำตอบ:\n{current_response}\n\nพยายามหา:\n1. ข้อผิดพลาดทางตรรกะ (Logical Fallacies)\n2. สมมติฐานที่ไม่มีหลักฐาน (Unsupported Assumptions)\n3. ข้อมูลที่อาจไม่ถูกต้อง (Potentially Incorrect Information)\n4. จุดอ่อนในการโต้แย้ง (Weak Arguments)\n5. สิ่งที่ขาดหายไป (Missing Elements)\n6. Bias ที่อาจมี (Potential Biases)\n\nเป็นนักวิจารณ์ที่เข้มงวด - อย่าประนีประนอม!\n\"\"\"\n        \n        critique_response = call_gemini(critique_prompt)\n        current_critique = critique_response\n        print_section(\"Critique\", current_critique[:500], \"⚔️\")\n        \n        results[\"rounds\"].append({\n            \"round\": round_num + 1,\n            \"response\": current_response,\n            \"critique\": current_critique\n        })\n        \n        time.sleep(1)\n    \n    # Final synthesis\n    synthesis_prompt = f\"\"\"\nหลังจากผ่าน {num_rounds} รอบของการวิพากษ์และปรับปรุง\n\nงาน:\n{task}\n\nสร้างคำตอบสุดท้ายที่:\n1. รวมจุดแข็งจากทุกรอบ\n2. หลีกเลี่ยงจุดอ่อนที่ถูกวิพากษ์\n3. มีความสมดุลและรอบคอบ\n\nคำตอบรอบ 1:\n{results['rounds'][0]['response'][:500]}\n\nวิพากษ์รอบ 1:\n{results['rounds'][0]['critique'][:500]}\n\nคำตอบรอบ 2:\n{results['rounds'][-1]['response'][:500]}\n\"\"\"\n    \n    final_response = call_gemini(synthesis_prompt)\n    results[\"final_response\"] = final_response\n    print_section(\"Final Response\", final_response, \"🏆\")\n    \n    return results\n\n#%%\n# ทดสอบ Self-Critique\ncontroversial_task = \"\"\"\nวิเคราะห์: \"บริษัทควรบังคับให้พนักงานกลับมาทำงานที่ออฟฟิศ 100% หรือไม่?\"\nพิจารณาทั้งมุมมองของนายจ้างและลูกจ้าง\n\"\"\"\n\nprint(\"⚔️ Starting Adversarial Self-Critique...\")\ncritique_results = adversarial_self_critique(controversial_task, num_rounds=2)\n\n#%% [markdown]\n# ---\n# # 📊 Part 9: Evaluation Methods (Extended)\n# **LLM-as-a-Judge, Reference-free, Automated Metrics & More**\n# \n# ---\n# ## 📖 ทฤษฎีเชิงลึก: ทำไมต้อง Evaluate?\n# \n# ```\n# ╔══════════════════════════════════════════════════════════════════════╗\n# ║              📊 WHY EVALUATION MATTERS                              ║\n# ╠══════════════════════════════════════════════════════════════════════╣\n# ║                                                                      ║\n# ║  \"If you can't measure it, you can't improve it.\"                   ║\n# ║  — Peter Drucker                                                     ║\n# ║                                                                      ║\n# ║  ❌ Without Evaluation:      ✅ With Evaluation:                     ║\n# ║  • ไม่รู้ว่าดีพอหรือยัง      • วัดคุณภาพเป็นตัวเลข                  ║\n# ║  • ไม่รู้ว่า prompt ไหนดีกว่า  • เปรียบเทียบเทคนิคได้                ║\n# ║  • ไม่รู้ว่า model ไหนดีกว่า  • ปรับปรุงอย่างเป็นระบบ               ║\n# ║  • ไม่รู้จุดอ่อนที่แท้จริง    • ค้นพบ failure modes                  ║\n# ║  • ไม่สามารถ scale ได้       • Automate quality assurance          ║\n# ║                                                                      ║\n# ╚══════════════════════════════════════════════════════════════════════╝\n# ```\n# \n# ### 📊 Evaluation Method Taxonomy\n# \n# ```\n# ┌──────────────────────────────────────────────────────────────────┐\n# │                  📊 EVALUATION TAXONOMY                          │\n# │                                                                   │\n# │  ┌─────────────────────────────────────────────────────────────┐ │\n# │  │  1. HUMAN EVALUATION (Gold Standard)                        │ │\n# │  │     • Expert review          • Crowdsourcing                │ │\n# │  │     • A/B testing            • User studies                 │ │\n# │  │     ✅ Most reliable  ❌ Expensive, slow, not scalable      │ │\n# │  └─────────────────────────────────────────────────────────────┘ │\n# │                                                                   │\n# │  ┌─────────────────────────────────────────────────────────────┐ │\n# │  │  2. LLM-BASED EVALUATION (Semi-automated)                  │ │\n# │  │     • LLM-as-a-Judge         • Pairwise Comparison          │ │\n# │  │     • Reference-free         • Rubric-based Scoring         │ │\n# │  │     • Multi-agent Debate     • Hallucination Detection      │ │\n# │  │     ✅ Scalable, consistent  ❌ May have biases             │ │\n# │  └─────────────────────────────────────────────────────────────┘ │\n# │                                                                   │\n# │  ┌─────────────────────────────────────────────────────────────┐ │\n# │  │  3. AUTOMATED METRICS (Fully automated)                     │ │\n# │  │     • BLEU, ROUGE, METEOR    • Semantic Similarity          │ │\n# │  │     • Perplexity             • F1 / Exact Match             │ │\n# │  │     • BERTScore              • Factual Consistency          │ │\n# │  │     ✅ Fast, cheap, reproducible  ❌ Limited aspects        │ │\n# │  └─────────────────────────────────────────────────────────────┘ │\n# │                                                                   │\n# │  ┌─────────────────────────────────────────────────────────────┐ │\n# │  │  4. TASK-SPECIFIC EVALUATION                                │ │\n# │  │     • Code: pass@k, execution accuracy                     │ │\n# │  │     • Math: exact match with ground truth                   │ │\n# │  │     • Classification: precision, recall, F1                 │ │\n# │  │     • Summarization: ROUGE-L, factual consistency           │ │\n# │  │     ✅ Most relevant  ❌ Need task-specific implementation  │ │\n# │  └─────────────────────────────────────────────────────────────┘ │\n# └──────────────────────────────────────────────────────────────────┘\n# ```\n# \n# ### 🔑 LLM-as-a-Judge: Known Biases\n# \n# ```\n# ╔══════════════════════════════════════════════════════════════╗\n# ║          ⚠️ LLM-AS-A-JUDGE KNOWN BIASES                    ║\n# ╠══════════════════════════════════════════════════════════════╣\n# ║                                                              ║\n# ║  1. 📐 Verbosity Bias                                       ║\n# ║     LLM มักให้คะแนนคำตอบที่ยาวกว่าดีกว่า                   ║\n# ║     → แก้: กำหนด \"ความยาวไม่ใช่เกณฑ์\"                      ║\n# ║                                                              ║\n# ║  2. 🔢 Position Bias                                        ║\n# ║     ใน pairwise comparison มักเลือกตัวเลือกแรก             ║\n# ║     → แก้: สลับตำแหน่ง A/B แล้วเฉลี่ย                     ║\n# ║                                                              ║\n# ║  3. 🪞 Self-enhancement Bias                                ║\n# ║     LLM มักให้คะแนน output ของตัวเองสูง                    ║\n# ║     → แก้: ใช้ model อื่นเป็น judge                         ║\n# ║                                                              ║\n# ║  4. 📊 Anchoring Bias                                       ║\n# ║     คะแนนขึ้นกับ rubric/scale ที่ให้                        ║\n# ║     → แก้: ใช้ detailed rubric พร้อมตัวอย่าง               ║\n# ║                                                              ║\n# ║  5. 🎭 Sycophancy                                           ║\n# ║     LLM อาจ \"เอาใจ\" ผู้ถามมากเกินไป                       ║\n# ║     → แก้: บอกให้ \"be critical and honest\"                  ║\n# ║                                                              ║\n# ║  Mitigation Strategies:                                      ║\n# ║  ✅ ใช้ detailed rubric + ตัวอย่างคะแนน                     ║\n# ║  ✅ สลับตำแหน่งใน pairwise comparison                       ║\n# ║  ✅ ใช้หลาย judges แล้วเฉลี่ย                               ║\n# ║  ✅ เปรียบเทียบกับ human evaluation                          ║\n# ╚══════════════════════════════════════════════════════════════╝\n# ```\n\n#%%\n# === Lab 9.1: LLM-as-a-Judge (Enhanced) ===\n\ndef llm_as_judge(task: str, response: str, rubric: Dict[str, str] = None) -> Dict:\n    \"\"\"\n    ใช้ LLM เป็นผู้ตัดสินคุณภาพของ response (Enhanced version)\n    \"\"\"\n    \n    if rubric is None:\n        rubric = {\n            \"relevance\": \"คำตอบตรงประเด็นกับคำถามมากน้อยเพียงใด\",\n            \"accuracy\": \"ข้อมูลในคำตอบถูกต้องมากน้อยเพียงใด\",\n            \"completeness\": \"คำตอบครบถ้วนสมบูรณ์มากน้อยเพียงใด\",\n            \"clarity\": \"คำตอบชัดเจน เข้าใจง่ายมากน้อยเพียงใด\",\n            \"helpfulness\": \"คำตอบมีประโยชน์ต่อผู้ถามมากน้อยเพียงใด\"\n        }\n    \n    rubric_text = \"\\n\".join([f\"- {k}: {v}\" for k, v in rubric.items()])\n    \n    judge_prompt = f\"\"\"\nคุณเป็นผู้ตัดสินที่เป็นกลางและเข้มงวด ประเมินคำตอบต่อไปนี้:\n\n=== คำถาม/งาน ===\n{task}\n\n=== คำตอบที่ต้องประเมิน ===\n{response}\n\n=== เกณฑ์การประเมิน ===\n{rubric_text}\n\nกรุณาประเมินโดย:\n1. ให้คะแนนแต่ละเกณฑ์ (1-10)\n2. อธิบายเหตุผลของคะแนน\n3. ให้คะแนนรวม (1-10)\n4. สรุปจุดแข็งและจุดอ่อน\n5. ให้ข้อเสนอแนะในการปรับปรุง\n\n⚠️ สำคัญ: ความยาวของคำตอบไม่ใช่ตัวชี้วัดคุณภาพ\nให้ประเมินจากเนื้อหาจริงเท่านั้น\n\nFormat:\nSCORES:\n- relevance: [score]/10 - [reason]\n- accuracy: [score]/10 - [reason]\n- completeness: [score]/10 - [reason]\n- clarity: [score]/10 - [reason]\n- helpfulness: [score]/10 - [reason]\n\nOVERALL: [score]/10\n\nSTRENGTHS:\n- ...\n\nWEAKNESSES:\n- ...\n\nSUGGESTIONS:\n- ...\n\"\"\"\n    \n    judge_response = call_gemini(judge_prompt)\n    \n    # Parse scores\n    result = {\n        \"task\": task,\n        \"response\": response,\n        \"evaluation\": judge_response,\n        \"scores\": {}\n    }\n    \n    # Extract individual scores\n    for key in rubric.keys():\n        score_match = re.search(rf'{key}:\\s*(\\d+(?:\\.\\d+)?)/10', judge_response)\n        if score_match:\n            result[\"scores\"][key] = float(score_match.group(1))\n    \n    # Extract overall score\n    overall_match = re.search(r'OVERALL:\\s*(\\d+(?:\\.\\d+)?)/10', judge_response)\n    if overall_match:\n        result[\"overall_score\"] = float(overall_match.group(1))\n    elif result[\"scores\"]:\n        result[\"overall_score\"] = sum(result[\"scores\"].values()) / len(result[\"scores\"])\n    \n    return result\n\n#%%\n# ทดสอบ LLM-as-a-Judge\n\ntest_task = \"อธิบายหลักการทำงานของ Blockchain แบบเข้าใจง่าย\"\n\ntest_response = \"\"\"\nBlockchain คือระบบบันทึกข้อมูลแบบกระจายศูนย์ เปรียบเหมือนสมุดบัญชีที่ทุกคนถือสำเนาเหมือนกัน\n\nหลักการทำงาน:\n1. เมื่อมี transaction ใหม่ ข้อมูลจะถูกรวมเป็น \"block\"\n2. Block จะถูกส่งให้ทุกคนในเครือข่ายตรวจสอบ\n3. เมื่อตรวจสอบผ่าน block จะถูกเชื่อมต่อกับ block ก่อนหน้า\n4. การแก้ไขข้อมูลเดิมแทบเป็นไปไม่ได้ เพราะต้องแก้ทุก block ที่ตามมา\n\nข้อดี: โปร่งใส ปลอดภัย ไม่มีตัวกลาง\nข้อเสีย: ใช้พลังงานมาก ช้ากว่าระบบทั่วไป\n\"\"\"\n\nprint(\"⚖️ LLM-as-a-Judge Evaluation:\")\njudge_result = llm_as_judge(test_task, test_response)\nprint(judge_result[\"evaluation\"])\nif judge_result.get(\"scores\"):\n    print(f\"\\n📊 Parsed Scores: {judge_result['scores']}\")\nif judge_result.get(\"overall_score\"):\n    print(f\"📊 Overall Score: {judge_result['overall_score']}/10\")\n\n#%%\n# === Lab 9.2: Pairwise Comparison with Position Debiasing ===\n\ndef pairwise_comparison_debiased(task: str, response_a: str, response_b: str) -> Dict:\n    \"\"\"\n    เปรียบเทียบ 2 คำตอบ พร้อม position bias mitigation\n    โดยสลับตำแหน่ง A/B แล้วเฉลี่ยผล\n    \"\"\"\n    \n    def run_comparison(first: str, second: str, first_label: str, second_label: str) -> str:\n        compare_prompt = f\"\"\"\nคุณเป็นผู้ตัดสินที่เป็นกลาง เปรียบเทียบ 2 คำตอบต่อไปนี้:\n\n=== คำถาม/งาน ===\n{task}\n\n=== คำตอบ {first_label} ===\n{first}\n\n=== คำตอบ {second_label} ===\n{second}\n\nกรุณา:\n1. วิเคราะห์จุดแข็งและจุดอ่อนของแต่ละคำตอบ\n2. เปรียบเทียบในด้าน:\n   - ความถูกต้อง\n   - ความครบถ้วน\n   - ความชัดเจน\n   - ความเป็นประโยชน์\n3. ตัดสินว่าคำตอบไหนดีกว่า\n\n⚠️ ตัดสินจากคุณภาพเนื้อหา ไม่ใช่ตำแหน่ง\nความยาวไม่ใช่ตัวชี้วัดคุณภาพ\n\nWINNER: [{first_label}/{second_label}/Tie]\nREASON: [explanation]\n\"\"\"\n        return call_gemini(compare_prompt)\n    \n    # Run 1: A first, B second\n    result_ab = run_comparison(response_a, response_b, \"A\", \"B\")\n    \n    time.sleep(1)\n    \n    # Run 2: B first, A second (swapped positions)\n    result_ba = run_comparison(response_b, response_a, \"B\", \"A\")\n    \n    # Extract winners\n    winner_ab = \"A\" if \"WINNER: A\" in result_ab.upper() or \"WINNER:A\" in result_ab.upper() else \\\n                \"B\" if \"WINNER: B\" in result_ab.upper() or \"WINNER:B\" in result_ab.upper() else \"Tie\"\n    \n    winner_ba = \"B\" if \"WINNER: B\" in result_ba.upper() or \"WINNER:B\" in result_ba.upper() else \\\n                \"A\" if \"WINNER: A\" in result_ba.upper() or \"WINNER:A\" in result_ba.upper() else \"Tie\"\n    \n    # Determine final winner (consistent across both orderings)\n    if winner_ab == winner_ba:\n        final_winner = winner_ab\n        confidence = \"HIGH (consistent across both orderings)\"\n    else:\n        final_winner = \"Tie (inconsistent)\"\n        confidence = \"LOW (position bias detected)\"\n    \n    return {\n        \"task\": task,\n        \"run1_result\": result_ab,\n        \"run1_winner\": winner_ab,\n        \"run2_result\": result_ba,\n        \"run2_winner\": winner_ba,\n        \"final_winner\": final_winner,\n        \"confidence\": confidence\n    }\n\n#%%\n# ทดสอบ Pairwise Comparison (Debiased)\ncomparison_task = \"แนะนำวิธีประหยัดเงินสำหรับคนเริ่มต้นทำงาน\"\n\nresponse_a = \"\"\"\nวิธีประหยัดเงิน:\n1. ทำบัญชีรายรับรายจ่าย\n2. ตั้งเป้าออมเงิน 10-20% ของรายได้\n3. หลีกเลี่ยงซื้อของไม่จำเป็น\n4. ทำอาหารกินเองแทนซื้อข้าวนอกบ้าน\n5. ใช้ระบบขนส่งสาธารณะ\n\"\"\"\n\nresponse_b = \"\"\"\nสำหรับคนเริ่มต้นทำงาน แนะนำหลัก 50/30/20:\n- 50% สำหรับค่าใช้จ่ายจำเป็น (ค่าเช่า อาหาร เดินทาง)\n- 30% สำหรับ lifestyle (ท่องเที่ยว entertainment)\n- 20% สำหรับออม/ลงทุน\n\nTips เพิ่มเติม:\n1. เปิดบัญชีออมเงินแยกต่างหาก - ตั้ง auto transfer ทุกเดือน\n2. ใช้แอป track ค่าใช้จ่าย เช่น Money Lover\n3. รอ 24 ชั่วโมงก่อนซื้อของราคาแพง\n4. หา side income เพิ่ม\n5. ศึกษาเรื่องการลงทุนพื้นฐาน\n\nเป้าหมาย: มี Emergency Fund 3-6 เดือนภายใน 1 ปี\n\"\"\"\n\nprint(\"🔄 Debiased Pairwise Comparison:\")\ncomparison_result = pairwise_comparison_debiased(comparison_task, response_a, response_b)\nprint(f\"\\n📊 Run 1 Winner: {comparison_result['run1_winner']}\")\nprint(f\"📊 Run 2 Winner: {comparison_result['run2_winner']}\")\nprint(f\"🏆 Final Winner: {comparison_result['final_winner']}\")\nprint(f\"🎯 Confidence: {comparison_result['confidence']}\")\n\n#%%\n# === Lab 9.3: Reference-free Evaluation ===\n\ndef reference_free_evaluation(task: str, response: str, evaluation_aspects: List[str] = None) -> Dict:\n    \"\"\"\n    ประเมินคุณภาพโดยไม่ต้องมีคำตอบอ้างอิง\n    \"\"\"\n    \n    if evaluation_aspects is None:\n        evaluation_aspects = [\n            \"Fluency (ความลื่นไหลของภาษา)\",\n            \"Coherence (ความเชื่อมโยงของเนื้อหา)\",\n            \"Informativeness (ความให้ข้อมูล)\",\n            \"Factual Consistency (ความสอดคล้องของข้อเท็จจริง — ไม่ขัดแย้งกันเอง)\",\n            \"No Hallucination (ไม่มีการสร้างข้อมูลเท็จ)\"\n        ]\n    \n    aspects_text = \"\\n\".join([f\"- {a}\" for a in evaluation_aspects])\n    \n    eval_prompt = f\"\"\"\nประเมินคุณภาพของ text ต่อไปนี้โดยไม่ต้องมีคำตอบอ้างอิง:\n\n=== Task/Context ===\n{task}\n\n=== Text to Evaluate ===\n{response}\n\n=== Evaluation Aspects ===\n{aspects_text}\n\nสำหรับแต่ละ aspect:\n1. ให้คะแนน 1-10\n2. ยกตัวอย่างประกอบจาก text\n3. ระบุปัญหาที่พบ (ถ้ามี)\n\nเพิ่มเติม:\n- ตรวจสอบ potential hallucinations (ข้อมูลที่อาจไม่จริง)\n- ระบุ claims ที่ต้องการ verification\n- ตรวจสอบ internal consistency (ข้อมูลไม่ขัดแย้งกันเอง)\n\nFormat:\n[Aspect 1]: [score]/10\n- Evidence: [quote from text]\n- Issues: [problems found]\n\n...\n\nPOTENTIAL HALLUCINATIONS:\n- [list suspicious claims]\n\nCLAIMS NEEDING VERIFICATION:\n- [list claims that should be fact-checked]\n\nINTERNAL CONSISTENCY: [PASS/FAIL] — [explanation]\n\nOVERALL QUALITY: [score]/10\nSUMMARY: [brief summary]\n\"\"\"\n    \n    response_eval = call_gemini(eval_prompt)\n    \n    return {\n        \"task\": task,\n        \"response\": response,\n        \"reference_free_evaluation\": response_eval\n    }\n\n#%%\n# ทดสอบ Reference-free Evaluation\ntest_text_task = \"เขียนบทความเกี่ยวกับประวัติศาสตร์ของ Bitcoin\"\n\ntest_text = \"\"\"\nBitcoin ถูกสร้างขึ้นในปี 2009 โดย Satoshi Nakamoto ซึ่งเป็นนามแฝง \nไม่มีใครรู้ตัวตนที่แท้จริง\n\nประวัติสำคัญ:\n- 2008: เผยแพร่ whitepaper\n- 2009: ขุด block แรก (Genesis Block) \n- 2010: มีการซื้อพิซซ่า 2 ถาดด้วย 10,000 BTC (มูลค่าปัจจุบันหลายพันล้าน)\n- 2017: ราคาพุ่งแตะ $20,000 ครั้งแรก\n- 2021: ราคาทำ all-time high ที่ $69,000\n- 2024: มีการอนุมัติ Bitcoin ETF ในสหรัฐฯ\n\nBitcoin ใช้เทคโนโลยี Blockchain และมี supply จำกัดที่ 21 ล้านเหรียญ\nทำให้หลายคนมองว่าเป็น \"digital gold\"\n\"\"\"\n\nprint(\"📋 Reference-free Evaluation:\")\nref_free_result = reference_free_evaluation(test_text_task, test_text)\nprint(ref_free_result[\"reference_free_evaluation\"])\n\n#%%\n# === Lab 9.4: Automated Metrics — BLEU-like & ROUGE-like Scores ===\n# (Simplified implementation without external dependencies)\n\ndef simple_ngrams(text: str, n: int) -> List[Tuple]:\n    \"\"\"สร้าง n-grams จาก text\"\"\"\n    words = text.lower().split()\n    return [tuple(words[i:i+n]) for i in range(len(words)-n+1)]\n\ndef simple_bleu(reference: str, candidate: str, max_n: int = 4) -> Dict:\n    \"\"\"\n    Simplified BLEU score calculation\n    BLEU measures how many n-grams in candidate appear in reference\n    \n    ใช้สำหรับ: การแปลภาษา, งานที่มี reference answer\n    \"\"\"\n    scores = {}\n    \n    for n in range(1, max_n + 1):\n        ref_ngrams = simple_ngrams(reference, n)\n        cand_ngrams = simple_ngrams(candidate, n)\n        \n        if not cand_ngrams:\n            scores[f\"bleu-{n}\"] = 0.0\n            continue\n        \n        ref_counter = Counter(ref_ngrams)\n        matches = 0\n        for ng in cand_ngrams:\n            if ref_counter[ng] > 0:\n                matches += 1\n                ref_counter[ng] -= 1\n        \n        scores[f\"bleu-{n}\"] = matches / len(cand_ngrams)\n    \n    # Brevity penalty\n    ref_len = len(reference.split())\n    cand_len = len(candidate.split())\n    bp = min(1.0, math.exp(1 - ref_len / cand_len)) if cand_len > 0 else 0\n    \n    # Geometric mean of n-gram precisions\n    valid_scores = [s for s in scores.values() if s > 0]\n    if valid_scores:\n        geo_mean = math.exp(sum(math.log(s) for s in valid_scores) / len(valid_scores))\n    else:\n        geo_mean = 0\n    \n    scores[\"bleu_combined\"] = bp * geo_mean\n    scores[\"brevity_penalty\"] = bp\n    \n    return scores\n\ndef simple_rouge_l(reference: str, candidate: str) -> Dict:\n    \"\"\"\n    Simplified ROUGE-L score (Longest Common Subsequence)\n    ROUGE measures how many n-grams in reference appear in candidate\n    \n    ใช้สำหรับ: Summarization\n    \"\"\"\n    ref_words = reference.lower().split()\n    cand_words = candidate.lower().split()\n    \n    # LCS using dynamic programming\n    m, n = len(ref_words), len(cand_words)\n    dp = [[0] * (n + 1) for _ in range(m + 1)]\n    \n    for i in range(1, m + 1):\n        for j in range(1, n + 1):\n            if ref_words[i-1] == cand_words[j-1]:\n                dp[i][j] = dp[i-1][j-1] + 1\n            else:\n                dp[i][j] = max(dp[i-1][j], dp[i][j-1])\n    \n    lcs_length = dp[m][n]\n    \n    precision = lcs_length / n if n > 0 else 0\n    recall = lcs_length / m if m > 0 else 0\n    f1 = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0\n    \n    return {\n        \"rouge_l_precision\": precision,\n        \"rouge_l_recall\": recall,\n        \"rouge_l_f1\": f1,\n        \"lcs_length\": lcs_length\n    }\n\ndef simple_rouge_n(reference: str, candidate: str, n: int = 1) -> Dict:\n    \"\"\"\n    Simplified ROUGE-N score\n    \"\"\"\n    ref_ngrams = simple_ngrams(reference, n)\n    cand_ngrams = simple_ngrams(candidate, n)\n    \n    ref_counter = Counter(ref_ngrams)\n    cand_counter = Counter(cand_ngrams)\n    \n    # Count overlapping n-grams\n    overlap = 0\n    for ng in cand_counter:\n        overlap += min(cand_counter[ng], ref_counter[ng])\n    \n    precision = overlap / len(cand_ngrams) if cand_ngrams else 0\n    recall = overlap / len(ref_ngrams) if ref_ngrams else 0\n    f1 = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0\n    \n    return {\n        f\"rouge_{n}_precision\": precision,\n        f\"rouge_{n}_recall\": recall,\n        f\"rouge_{n}_f1\": f1\n    }\n\n#%%\n# ทดสอบ Automated Metrics\nreference_text = \"Bitcoin ถูกสร้างขึ้นในปี 2009 โดยบุคคลนามแฝง Satoshi Nakamoto ใช้เทคโนโลยี Blockchain ที่มีความปลอดภัยสูง\"\ncandidate_text = \"Bitcoin สร้างขึ้นปี 2009 โดย Satoshi Nakamoto ใช้ Blockchain เป็นเทคโนโลยีหลัก มีความปลอดภัย\"\n\nprint(\"📊 Automated Metrics:\")\nprint(\"=\" * 50)\n\nbleu_scores = simple_bleu(reference_text, candidate_text)\nprint(\"BLEU Scores:\")\nfor k, v in bleu_scores.items():\n    print(f\"  {k}: {v:.4f}\")\n\nprint()\n\nrouge_l = simple_rouge_l(reference_text, candidate_text)\nprint(\"ROUGE-L Scores:\")\nfor k, v in rouge_l.items():\n    print(f\"  {k}: {v:.4f}\" if isinstance(v, float) else f\"  {k}: {v}\")\n\nprint()\n\nrouge_1 = simple_rouge_n(reference_text, candidate_text, n=1)\nprint(\"ROUGE-1 Scores:\")\nfor k, v in rouge_1.items():\n    print(f\"  {k}: {v:.4f}\")\n\nrouge_2 = simple_rouge_n(reference_text, candidate_text, n=2)\nprint(\"\\nROUGE-2 Scores:\")\nfor k, v in rouge_2.items():\n    print(f\"  {k}: {v:.4f}\")\n\n#%% [markdown]\n# ### 📖 ทำความเข้าใจ Automated Metrics\n# \n# ```\n# ╔══════════════════════════════════════════════════════════════════╗\n# ║               📊 UNDERSTANDING AUTOMATED METRICS                ║\n# ╠══════════════════════════════════════════════════════════════════╣\n# ║                                                                  ║\n# ║  📘 BLEU (Bilingual Evaluation Understudy):                     ║\n# ║  • วัด: n-gram ของ candidate ที่ตรงกับ reference                ║\n# ║  • มุมมอง: Precision-focused                                    ║\n# ║  • ดูว่า: \"output มีคำ/วลีจาก reference มากเท่าไหร่\"          ║\n# ║  • ใช้กับ: Machine Translation, Text Generation                ║\n# ║  • Score Range: 0.0 - 1.0 (สูง = ดี)                          ║\n# ║                                                                  ║\n# ║  📗 ROUGE (Recall-Oriented Understudy for Gisting Evaluation):  ║\n# ║  • วัด: n-gram ของ reference ที่ปรากฏใน candidate              ║\n# ║  • มุมมอง: Recall-focused                                      ║\n# ║  • ดูว่า: \"reference ถูกครอบคลุมมากเท่าไหร่\"                   ║\n# ║  • ใช้กับ: Summarization                                        ║\n# ║  • Variants: ROUGE-1, ROUGE-2, ROUGE-L                        ║\n# ║                                                                  ║\n# ║  ⚖️ เปรียบเทียบ:                                                ║\n# ║  ┌─────────────────────────────────────────────────────┐       ║\n# ║  │ Reference: \"แมวสีดำนอนบนเสื่อ\"                       │       ║\n# ║  │ Candidate: \"แมวดำนอนอยู่บนเสื่อสีน้ำตาล\"              │       ║\n# ║  │                                                       │       ║\n# ║  │ BLEU (Precision): candidate มีคำจาก ref กี่คำ?       │       ║\n# ║  │ → \"แมว\", \"นอน\", \"บน\", \"เสื่อ\" = 4/6 ≈ 0.67        │       ║\n# ║  │                                                       │       ║\n# ║  │ ROUGE (Recall): ref ถูกครอบคลุมกี่คำ?                │       ║\n# ║  │ → \"แมว\", \"นอน\", \"บน\", \"เสื่อ\" = 4/5 = 0.80        │       ║\n# ║  └─────────────────────────────────────────────────────┘       ║\n# ║                                                                  ║\n# ║  ⚠️ Limitations:                                                ║\n# ║  • ไม่เข้าใจ meaning (synonyms ไม่ match)                      ║\n# ║  • ไม่เหมาะกับ open-ended tasks                                ║\n# ║  • ต้องมี reference (ground truth)                              ║\n# ║  → ใช้ร่วมกับ LLM-based evaluation เพื่อผลที่สมบูรณ์          ║\n# ╚══════════════════════════════════════════════════════════════════╝\n# ```\n\n#%%\n# === Lab 9.5: Semantic Similarity (LLM-based) ===\n\ndef semantic_similarity_eval(text_a: str, text_b: str) -> Dict:\n    \"\"\"\n    ใช้ LLM ประเมิน semantic similarity ระหว่าง 2 texts\n    (ใช้แทน embedding-based similarity เมื่อไม่มี embedding model)\n    \"\"\"\n    \n    eval_prompt = f\"\"\"\nประเมินความคล้ายคลึงทาง **ความหมาย** (semantic similarity) ของ 2 ข้อความต่อไปนี้:\n\n=== Text A ===\n{text_a}\n\n=== Text B ===\n{text_b}\n\nประเมินใน 4 มิติ:\n1. **Topical Similarity** (หัวข้อเดียวกัน): 0.0 - 1.0\n2. **Information Overlap** (ข้อมูลซ้อนทับ): 0.0 - 1.0\n3. **Sentiment Alignment** (ความรู้สึกไปทางเดียวกัน): 0.0 - 1.0\n4. **Intent Match** (เจตนาตรงกัน): 0.0 - 1.0\n\nOverall Semantic Similarity: (เฉลี่ยถ่วงน้ำหนัก)\n\nFormat:\ntopical_similarity: [score]\ninformation_overlap: [score]\nsentiment_alignment: [score]\nintent_match: [score]\noverall_similarity: [score]\nexplanation: [brief explanation]\n\"\"\"\n    \n    result = call_gemini(eval_prompt)\n    \n    # Extract overall similarity\n    sim_match = re.search(r'overall_similarity:\\s*([\\d.]+)', result)\n    overall_sim = float(sim_match.group(1)) if sim_match else None\n    \n    return {\n        \"text_a\": text_a[:100],\n        \"text_b\": text_b[:100],\n        \"evaluation\": result,\n        \"overall_similarity\": overall_sim\n    }\n\n#%%\n# ทดสอบ Semantic Similarity\ntext_a = \"AI กำลังเปลี่ยนแปลงวิธีการทำงาน หลายอาชีพอาจถูกแทนที่ด้วยระบบอัตโนมัติ\"\ntext_b = \"ปัญญาประดิษฐ์ส่งผลกระทบต่อตลาดแรงงาน งานบางประเภทอาจหายไปเนื่องจาก automation\"\n\nprint(\"🔗 Semantic Similarity Evaluation:\")\nsim_result = semantic_similarity_eval(text_a, text_b)\nprint(sim_result[\"evaluation\"])\nprint(f\"\\n📊 Overall Similarity: {sim_result['overall_similarity']}\")\n\n#%%\n# === Lab 9.6: Hallucination Detection (Dedicated) ===\n\ndef hallucination_detector(topic: str, text: str) -> Dict:\n    \"\"\"\n    ตรวจจับ Hallucination โดยเฉพาะ\n    \"\"\"\n    \n    detection_prompt = f\"\"\"\nคุณเป็นผู้เชี่ยวชาญด้านการตรวจสอบข้อเท็จจริง (Fact-checker)\n\n=== หัวข้อ ===\n{topic}\n\n=== ข้อความที่ต้องตรวจสอบ ===\n{text}\n\nกรุณาตรวจสอบทุก factual claim ในข้อความ:\n\nสำหรับแต่ละ claim ที่พบ:\n1. ระบุ claim\n2. ประเมินความน่าเชื่อถือ:\n   - ✅ VERIFIED: มั่นใจว่าถูกต้อง\n   - ⚠️ UNCERTAIN: ไม่แน่ใจ ต้องตรวจสอบ\n   - ❌ LIKELY FALSE: น่าจะไม่ถูกต้อง\n   - 🔍 UNVERIFIABLE: ไม่สามารถตรวจสอบได้\n3. อธิบายเหตุผล\n\nFormat:\nCLAIM 1: \"[claim text]\"\nSTATUS: [✅/⚠️/❌/🔍]\nREASON: [explanation]\nCORRECTION: [if false, provide correct info]\n\n...\n\nSUMMARY:\n- Total claims found: [N]\n- Verified: [N]\n- Uncertain: [N]\n- Likely False: [N]\n- Unverifiable: [N]\n\nHALLUCINATION_RISK: [LOW/MEDIUM/HIGH/CRITICAL]\nOVERALL_TRUSTWORTHINESS: [score]/10\n\"\"\"\n    \n    result = call_gemini(detection_prompt)\n    \n    # Extract risk level\n    risk_match = re.search(r'HALLUCINATION_RISK:\\s*(LOW|MEDIUM|HIGH|CRITICAL)', result)\n    risk_level = risk_match.group(1) if risk_match else \"UNKNOWN\"\n    \n    return {\n        \"topic\": topic,\n        \"text\": text,\n        \"analysis\": result,\n        \"hallucination_risk\": risk_level\n    }\n\n#%%\n# ทดสอบ Hallucination Detection\ntest_topic = \"ประวัติศาสตร์ของประเทศไทย\"\ntest_text_with_errors = \"\"\"\nประเทศไทยไม่เคยตกเป็นอาณานิคมของชาติตะวันตก เป็นประเทศเดียวในเอเชียตะวันออกเฉียงใต้ที่รักษาเอกราชไว้ได้\nกรุงเทพมหานครก่อตั้งโดยพระบาทสมเด็จพระพุทธยอดฟ้าจุฬาโลกมหาราช ในปี พ.ศ. 2325\nประเทศไทยเปลี่ยนการปกครองเป็นระบอบประชาธิปไตยในปี พ.ศ. 2475\nปัจจุบันประเทศไทยมีประชากรประมาณ 90 ล้านคน\n\"\"\"\n\nprint(\"🔎 Hallucination Detection:\")\nhallucination_result = hallucination_detector(test_topic, test_text_with_errors)\nprint(hallucination_result[\"analysis\"])\nprint(f\"\\n⚠️ Hallucination Risk: {hallucination_result['hallucination_risk']}\")\n\n#%%\n# === Lab 9.7: Multi-Agent Debate Evaluation ===\n\ndef multi_agent_debate_eval(task: str, response: str, num_agents: int = 3) -> Dict:\n    \"\"\"\n    ใช้หลาย \"agents\" (persona) ประเมินคำตอบเดียวกันจากมุมมองต่างกัน\n    แล้ว synthesize ผลลัพธ์\n    \"\"\"\n    \n    agent_personas = [\n        {\n            \"name\": \"Technical Expert\",\n            \"prompt\": \"คุณเป็นผู้เชี่ยวชาญด้านเทคนิค ดูความถูกต้องทางเทคนิค ความแม่นยำ และ technical depth\"\n        },\n        {\n            \"name\": \"Communication Specialist\", \n            \"prompt\": \"คุณเป็นผู้เชี่ยวชาญด้านสื่อสาร ดูความชัดเจน การจัดระเบียบ และว่าผู้อ่านทั่วไปจะเข้าใจหรือไม่\"\n        },\n        {\n            \"name\": \"Devil's Advocate\",\n            \"prompt\": \"คุณเป็นผู้คัดค้าน ดูช่องโหว่ สมมติฐานที่ผิดพลาด bias และสิ่งที่ขาดหาย พยายามหาจุดอ่อน\"\n        }\n    ]\n    \n    agent_evaluations = []\n    \n    for i, agent in enumerate(agent_personas[:num_agents]):\n        eval_prompt = f\"\"\"\n{agent['prompt']}\n\nประเมินคำตอบนี้:\n\nTask: {task}\nResponse: {response}\n\nให้คะแนน 1-10 และอธิบายจากมุมมองของคุณ\nระบุจุดแข็ง จุดอ่อน และข้อเสนอแนะ\n\nSCORE: [1-10]\nSTRENGTHS: [list]\nWEAKNESSES: [list]\nSUGGESTIONS: [list]\n\"\"\"\n        \n        result = call_gemini(eval_prompt)\n        agent_evaluations.append({\n            \"agent\": agent[\"name\"],\n            \"evaluation\": result\n        })\n        \n        print(f\"\\n👤 {agent['name']}:\")\n        print(result[:300] + \"...\" if len(result) > 300 else result)\n        time.sleep(1)\n    \n    # Synthesize\n    all_evals = \"\\n\\n\".join([f\"[{ae['agent']}]\\n{ae['evaluation']}\" for ae in agent_evaluations])\n    \n    synthesis_prompt = f\"\"\"\nสรุปผลการประเมินจาก {num_agents} ผู้เชี่ยวชาญ:\n\n{all_evals}\n\nสร้างสรุป:\n1. คะแนนเฉลี่ยจากทุกผู้เชี่ยวชาญ\n2. จุดที่เห็นพ้องกัน (ทั้งจุดแข็งและจุดอ่อน)\n3. จุดที่ขัดแย้งกัน\n4. ข้อเสนอแนะสุดท้าย\n\nCONSENSUS_SCORE: [1-10]\n\"\"\"\n    \n    synthesis = call_gemini(synthesis_prompt)\n    \n    return {\n        \"task\": task,\n        \"response\": response,\n        \"agent_evaluations\": agent_evaluations,\n        \"synthesis\": synthesis\n    }\n\n#%%\n# ทดสอบ Multi-Agent Debate\ndebate_task = \"อธิบาย Quantum Computing สำหรับผู้เริ่มต้น\"\ndebate_response = \"\"\"\nQuantum Computing ใช้ qubit แทน bit ปกติ ทำให้คำนวณได้เร็วกว่าหลายล้านเท่า\nQubit สามารถเป็นทั้ง 0 และ 1 พร้อมกัน (superposition)\nGoogle มี quantum computer ที่ทำงานเร็วกว่า supercomputer ทั่วไป\nในอนาคต quantum computing จะทำให้ encryption ทั้งหมดไม่ปลอดภัย\n\"\"\"\n\nprint(\"🤝 Multi-Agent Debate Evaluation:\")\ndebate_result = multi_agent_debate_eval(debate_task, debate_response, num_agents=3)\nprint(\"\\n📊 Synthesis:\")\nprint(debate_result[\"synthesis\"])\n\n#%% [markdown]\n# ---\n# # 📊 Part 10: Comprehensive Evaluation Framework\n# **รวมทุกวิธี Evaluation เข้าด้วยกัน**\n# \n# ---\n# \n# ### 📊 Evaluation Framework Architecture\n# \n# ```\n# ╔══════════════════════════════════════════════════════════════════╗\n# ║          🏗️ COMPREHENSIVE EVALUATION ARCHITECTURE               ║\n# ╠══════════════════════════════════════════════════════════════════╣\n# ║                                                                  ║\n# ║  Input: (task, response)                                        ║\n# ║         │                                                        ║\n# ║         ├──▶ [LLM-as-Judge] ──▶ Multi-aspect scores            ║\n# ║         │                                                        ║\n# ║         ├──▶ [Reference-free] ──▶ Quality assessment            ║\n# ║         │                                                        ║\n# ║         ├──▶ [Hallucination Check] ──▶ Risk level              ║\n# ║         │                                                        ║\n# ║         ├──▶ [Automated Metrics] ──▶ BLEU/ROUGE (if ref)       ║\n# ║         │                                                        ║\n# ║         ├──▶ [Multi-Agent Debate] ──▶ Diverse perspectives     ║\n# ║         │                                                        ║\n# ║         └──▶ [Pairwise Compare] ──▶ Relative ranking           ║\n# ║                     │                                            ║\n# ║                     ▼                                            ║\n# ║         ┌─────────────────────────┐                             ║\n# ║         │  📊 AGGREGATION LAYER   │                             ║\n# ║         │  • Weighted average     │                             ║\n# ║         │  • Confidence interval  │                             ║\n# ║         │  • Risk assessment      │                             ║\n# ║         └──────────┬──────────────┘                             ║\n# ║                    ▼                                             ║\n# ║         ┌─────────────────────────┐                             ║\n# ║         │  📋 EVALUATION REPORT   │                             ║\n# ║         │  • Overall score        │                             ║\n# ║         │  • Dimension scores     │                             ║\n# ║         │  • Risk flags           │                             ║\n# ║         │  • Improvement areas    │                             ║\n# ║         │  • Recommendations      │                             ║\n# ║         └─────────────────────────┘                             ║\n# ╚══════════════════════════════════════════════════════════════════╝\n# ```\n\n#%%\n# === Lab 10.1: Complete Evaluation Pipeline ===\n\nclass ComprehensiveEvaluator:\n    \"\"\"\n    Evaluation Framework ที่รวมหลายวิธีเข้าด้วยกัน\n    \"\"\"\n    \n    def __init__(self, model_name: str):\n        self.model_name = model_name\n    \n    def evaluate(self, task: str, response: str, \n                 reference: str = None,\n                 include_pairwise: bool = False, \n                 alternative_response: str = None,\n                 include_multi_agent: bool = False) -> Dict:\n        \"\"\"\n        ประเมินแบบครบวงจร\n        \"\"\"\n        results = {\n            \"task\": task,\n            \"response\": response,\n            \"evaluations\": {},\n            \"flags\": []\n        }\n        \n        print(\"🔍 Starting Comprehensive Evaluation...\")\n        print(\"=\" * 60)\n        \n        # 1. LLM-as-a-Judge (Multi-aspect)\n        print(\"\\n📊 1. LLM-as-a-Judge Evaluation\")\n        judge_result = llm_as_judge(task, response)\n        results[\"evaluations\"][\"llm_judge\"] = judge_result\n        score = judge_result.get('overall_score', 'N/A')\n        print(f\"   Score: {score}/10\")\n        if isinstance(score, (int, float)) and score < 5:\n            results[\"flags\"].append(\"⚠️ LOW QUALITY: LLM Judge score < 5\")\n        \n        time.sleep(1)\n        \n        # 2. Reference-free Evaluation\n        print(\"\\n📋 2. Reference-free Evaluation\")\n        ref_free_result = reference_free_evaluation(task, response)\n        results[\"evaluations\"][\"reference_free\"] = ref_free_result\n        \n        time.sleep(1)\n        \n        # 3. Hallucination Check\n        print(\"\\n🔎 3. Hallucination Detection\")\n        hallucination_result = hallucination_detector(task, response)\n        results[\"evaluations\"][\"hallucination_check\"] = hallucination_result\n        if hallucination_result[\"hallucination_risk\"] in [\"HIGH\", \"CRITICAL\"]:\n            results[\"flags\"].append(f\"🚨 HALLUCINATION RISK: {hallucination_result['hallucination_risk']}\")\n        \n        time.sleep(1)\n        \n        # 4. Automated Metrics (if reference provided)\n        if reference:\n            print(\"\\n📏 4. Automated Metrics (BLEU/ROUGE)\")\n            bleu = simple_bleu(reference, response)\n            rouge_l = simple_rouge_l(reference, response)\n            rouge_1 = simple_rouge_n(reference, response, n=1)\n            results[\"evaluations\"][\"automated_metrics\"] = {\n                \"bleu\": bleu,\n                \"rouge_l\": rouge_l,\n                \"rouge_1\": rouge_1\n            }\n            print(f\"   BLEU Combined: {bleu['bleu_combined']:.4f}\")\n            print(f\"   ROUGE-L F1: {rouge_l['rouge_l_f1']:.4f}\")\n            print(f\"   ROUGE-1 F1: {rouge_1['rouge_1_f1']:.4f}\")\n        \n        # 5. Pairwise Comparison (if alternative provided)\n        if include_pairwise and alternative_response:\n            print(\"\\n⚖️ 5. Pairwise Comparison (Debiased)\")\n            pairwise_result = pairwise_comparison_debiased(task, response, alternative_response)\n            results[\"evaluations\"][\"pairwise\"] = pairwise_result\n            print(f\"   Winner: {pairwise_result['final_winner']}\")\n            time.sleep(1)\n        \n        # 6. Multi-Agent Debate (optional)\n        if include_multi_agent:\n            print(\"\\n🤝 6. Multi-Agent Debate\")\n            debate_result = multi_agent_debate_eval(task, response, num_agents=3)\n            results[\"evaluations\"][\"multi_agent_debate\"] = debate_result\n        \n        # 7. Aggregate Score\n        results[\"aggregate\"] = self._calculate_aggregate(results)\n        \n        # 8. Generate Report\n        results[\"report\"] = self._generate_report(results)\n        \n        return results\n    \n    def _calculate_aggregate(self, results: Dict) -> Dict:\n        \"\"\"Calculate aggregate score from all evaluations\"\"\"\n        scores = []\n        \n        if \"llm_judge\" in results[\"evaluations\"]:\n            judge = results[\"evaluations\"][\"llm_judge\"]\n            if judge.get(\"overall_score\"):\n                scores.append((\"LLM Judge\", judge[\"overall_score\"]))\n        \n        if scores:\n            avg = sum(s[1] for s in scores) / len(scores)\n            return {\n                \"average_score\": round(avg, 2),\n                \"individual_scores\": scores,\n                \"num_evaluations\": len(scores),\n                \"flags_count\": len(results.get(\"flags\", []))\n            }\n        return {\"average_score\": None, \"num_evaluations\": 0, \"flags_count\": 0}\n    \n    def _generate_report(self, results: Dict) -> str:\n        \"\"\"Generate human-readable evaluation report\"\"\"\n        report = []\n        report.append(\"=\" * 60)\n        report.append(\"📋 COMPREHENSIVE EVALUATION REPORT\")\n        report.append(\"=\" * 60)\n        \n        agg = results.get(\"aggregate\", {})\n        report.append(f\"\\n📊 Overall Score: {agg.get('average_score', 'N/A')}/10\")\n        report.append(f\"📊 Evaluations Used: {agg.get('num_evaluations', 0)}\")\n        \n        flags = results.get(\"flags\", [])\n        if flags:\n            report.append(f\"\\n🚩 Flags ({len(flags)}):\")\n            for f in flags:\n                report.append(f\"   {f}\")\n        else:\n            report.append(\"\\n✅ No flags raised\")\n        \n        if \"hallucination_check\" in results[\"evaluations\"]:\n            risk = results[\"evaluations\"][\"hallucination_check\"][\"hallucination_risk\"]\n            report.append(f\"\\n🔎 Hallucination Risk: {risk}\")\n        \n        report.append(\"\\n\" + \"=\" * 60)\n        \n        return \"\\n\".join(report)\n\n#%%\n# ทดสอบ Comprehensive Evaluator\nevaluator = ComprehensiveEvaluator(MODEL_NAME)\n\ntest_task = \"อธิบายความแตกต่างระหว่าง Machine Learning กับ Deep Learning\"\n\ntest_response = \"\"\"\nMachine Learning (ML) และ Deep Learning (DL) มีความแตกต่างกันดังนี้:\n\nMachine Learning:\n- ใช้ algorithm แบบดั้งเดิม เช่น Decision Tree, SVM, Linear Regression\n- ต้อง feature engineering ด้วยมือ\n- ใช้ข้อมูลน้อยกว่า\n- เหมาะกับปัญหาที่ไม่ซับซ้อนมาก\n\nDeep Learning:\n- ใช้ Neural Networks หลายชั้น\n- สามารถเรียนรู้ features เอง\n- ต้องการข้อมูลมาก\n- เหมาะกับ image, text, speech\n\nDL เป็น subset ของ ML ที่พัฒนาขึ้นมาเพื่อจัดการกับปัญหาที่ซับซ้อนขึ้น\n\"\"\"\n\ntest_reference = \"\"\"\nMachine Learning เป็นสาขาของ AI ที่ใช้ algorithms เรียนรู้จากข้อมูล \nDeep Learning เป็น subset ของ ML ที่ใช้ neural networks หลายชั้น\nML ต้อง feature engineering แต่ DL เรียนรู้ features เอง\nDL ต้องการข้อมูลและ computing power มากกว่า\n\"\"\"\n\nevaluation_results = evaluator.evaluate(\n    test_task, \n    test_response,\n    reference=test_reference\n)\n\nprint(evaluation_results[\"report\"])\n\n#%% [markdown]\n# ---\n# # 🎯 Part 11: Final Challenge — Combined Techniques\n# **รวมทุกเทคนิคที่เรียนมา**\n# \n# ---\n# \n# ### 📊 Challenge Architecture\n# \n# ```\n# ╔══════════════════════════════════════════════════════════════════════╗\n# ║                 🏆 FINAL CHALLENGE PIPELINE                         ║\n# ╠══════════════════════════════════════════════════════════════════════╣\n# ║                                                                      ║\n# ║  Input: Complex Problem                                             ║\n# ║         │                                                            ║\n# ║  Stage 1: Few-shot CoT ──▶ Initial Analysis                        ║\n# ║         │                                                            ║\n# ║  Stage 2: Tree-of-Thought ──▶ Multiple Solutions                   ║\n# ║         │                                                            ║\n# ║  Stage 3: Reflection ──▶ Improved Plan                             ║\n# ║         │                                                            ║\n# ║  Stage 4: Self-Critique ──▶ Stress-tested Plan                     ║\n# ║         │                                                            ║\n# ║  Stage 5: Synthesis ──▶ Final Comprehensive Plan                   ║\n# ║         │                                                            ║\n# ║  Stage 6: Comprehensive Evaluation ──▶ Quality Report              ║\n# ║                                                                      ║\n# ╚══════════════════════════════════════════════════════════════════════╝\n# ```\n\n#%%\n# === Final Challenge: Build a Complete Analysis System ===\n\ndef advanced_analysis_system(problem: str) -> Dict:\n    \"\"\"\n    ระบบวิเคราะห์ที่รวมหลายเทคนิค:\n    1. Few-shot CoT สำหรับ initial analysis\n    2. Tree-of-Thought สำหรับสร้างทางเลือก\n    3. Reflection สำหรับปรับปรุง\n    4. Self-Critique สำหรับตรวจสอบ\n    5. Synthesis สำหรับคำตอบสุดท้าย\n    6. Comprehensive Evaluation สำหรับประเมิน\n    \"\"\"\n    \n    results = {\"stages\": {}}\n    \n    print(\"🚀 Starting Advanced Analysis System\")\n    print(\"=\" * 60)\n    \n    # Stage 1: Few-shot + CoT for initial analysis\n    print(\"\\n📌 Stage 1: Initial Analysis with Few-shot CoT\")\n    stage1_prompt = f\"\"\"\nวิเคราะห์ปัญหาต่อไปนี้โดยใช้ตัวอย่างเป็นแนวทาง:\n\nตัวอย่าง:\nปัญหา: ร้านค้าปลีกยอดขายลดลง\nวิเคราะห์:\n- ขั้นตอน 1: ระบุสาเหตุ → แข่งขันสูง, เศรษฐกิจ, พฤติกรรมผู้บริโภค\n- ขั้นตอน 2: ประเมินผลกระทบ → รายได้ลด, กำไรลด\n- ขั้นตอน 3: หาทางออก → ปรับกลยุทธ์, ลดต้นทุน\n\nปัญหาของคุณ:\n{problem}\n\nวิเคราะห์ทีละขั้นตอนอย่างละเอียด\n\"\"\"\n    \n    response1 = call_gemini(stage1_prompt)\n    results[\"stages\"][\"initial_analysis\"] = response1\n    print(response1[:500] + \"...\")\n    \n    time.sleep(1)\n    \n    # Stage 2: Tree-of-Thought for solutions\n    print(\"\\n📌 Stage 2: Tree-of-Thought for Solutions\")\n    stage2_prompt = f\"\"\"\nจากการวิเคราะห์:\n{results['stages']['initial_analysis'][:1000]}\n\nสร้าง 3 ทางเลือกในการแก้ปัญหา โดยแต่ละทางเลือกต้องมี:\n- แนวคิดหลัก\n- ข้อดี/ข้อเสีย\n- ความเป็นไปได้ (1-10)\n\nแล้วเลือกทางที่ดีที่สุดพร้อมเหตุผล\n\"\"\"\n    \n    response2 = call_gemini(stage2_prompt, temperature=0.5)\n    results[\"stages\"][\"solutions_tree\"] = response2\n    print(response2[:500] + \"...\")\n    \n    time.sleep(1)\n    \n    # Stage 3: Reflection and Improvement\n    print(\"\\n📌 Stage 3: Reflection & Improvement\")\n    stage3_prompt = f\"\"\"\nทบทวนทางออกที่เลือก:\n{results['stages']['solutions_tree'][:1000]}\n\n1. จุดแข็งของแผน\n2. จุดอ่อนที่ต้องแก้ไข\n3. ความเสี่ยงที่อาจเกิด\n4. ปรับปรุงแผนให้ดีขึ้น\n\"\"\"\n    \n    response3 = call_gemini(stage3_prompt)\n    results[\"stages\"][\"reflection\"] = response3\n    print(response3[:500] + \"...\")\n    \n    time.sleep(1)\n    \n    # Stage 4: Self-Critique\n    print(\"\\n📌 Stage 4: Self-Critique\")\n    stage4_prompt = f\"\"\"\nในฐานะนักวิจารณ์ที่เข้มงวด วิพากษ์แผนนี้:\n{results['stages']['reflection'][:1000]}\n\nหา:\n1. ข้อผิดพลาดทางตรรกะ\n2. สมมติฐานที่อาจผิด\n3. สิ่งที่ขาดหาย\n4. ความเสี่ยงที่มองข้าม\n\"\"\"\n    \n    response4 = call_gemini(stage4_prompt)\n    results[\"stages\"][\"critique\"] = response4\n    print(response4[:500] + \"...\")\n    \n    time.sleep(1)\n    \n    # Stage 5: Final Synthesis\n    print(\"\\n📌 Stage 5: Final Synthesis\")\n    stage5_prompt = f\"\"\"\nสังเคราะห์ทุกอย่างที่ได้:\n\nปัญหา: {problem}\n\nการวิเคราะห์: {results['stages']['initial_analysis'][:500]}\nทางออก: {results['stages']['solutions_tree'][:500]}\nการปรับปรุง: {results['stages']['reflection'][:500]}\nการวิพากษ์: {results['stages']['critique'][:500]}\n\nสร้างแผนสุดท้ายที่:\n1. แก้ไขทุกจุดอ่อนที่พบ\n2. มีความเป็นไปได้สูง\n3. มี action items ที่ชัดเจน\n4. มี timeline และ KPIs\n\"\"\"\n    \n    response5 = call_gemini(stage5_prompt)\n    results[\"stages\"][\"final_plan\"] = response5\n    print(response5)\n    \n    time.sleep(1)\n    \n    # Stage 6: Comprehensive Evaluation\n    print(\"\\n📌 Stage 6: Comprehensive Evaluation\")\n    eval_result = llm_as_judge(\n        problem, \n        results[\"stages\"][\"final_plan\"],\n        rubric={\n            \"feasibility\": \"แผนนี้เป็นไปได้จริงมากน้อยเพียงใด\",\n            \"completeness\": \"แผนครบถ้วน ครอบคลุมทุกด้านหรือไม่\",\n            \"practicality\": \"มี action items ที่ปฏิบัติได้จริงหรือไม่\",\n            \"innovation\": \"มีความคิดสร้างสรรค์ แตกต่างจากแนวทางทั่วไปหรือไม่\",\n            \"risk_management\": \"มีการจัดการความเสี่ยงที่ดีหรือไม่\"\n        }\n    )\n    results[\"evaluation\"] = eval_result\n    print(eval_result[\"evaluation\"])\n    \n    return results\n\n#%%\n# ทดสอบ Advanced Analysis System\ncomplex_problem = \"\"\"\nบริษัท Startup ด้าน EdTech มีแอป e-learning ที่มีผู้ใช้ 10,000 คน\nแต่ retention rate ต่ำมาก (เพียง 15% กลับมาใช้หลัง 1 เดือน)\nมีงบประมาณ 2 ล้านบาท และต้องเพิ่ม retention เป็น 40% ภายใน 6 เดือน\nทีมมี developer 3 คน และ designer 1 คน\n\"\"\"\n\nprint(\"🎯 Running Final Challenge...\")\nfinal_results = advanced_analysis_system(complex_problem)\n\n#%% [markdown]\n# ---\n# # 📝 Summary & Comprehensive Cheat Sheet\n# \n# ---\n# \n# ## 🗺️ Technique Selection Decision Tree\n# \n# ```\n# ╔══════════════════════════════════════════════════════════════════╗\n# ║         🎯 TECHNIQUE SELECTION DECISION TREE                    ║\n# ╠══════════════════════════════════════════════════════════════════╣\n# ║                                                                  ║\n# ║  What type of task?                                              ║\n# ║  │                                                               ║\n# ║  ├─▶ Simple format/classification → Few-shot ⭐                  ║\n# ║  │                                                               ║\n# ║  ├─▶ Requires reasoning/math?                                   ║\n# ║  │   ├─▶ Low stakes → CoT ⭐⭐                                   ║\n# ║  │   └─▶ High stakes → Self-Consistency CoT ⭐⭐⭐                ║\n# ║  │                                                               ║\n# ║  ├─▶ Complex multi-step task?                                   ║\n# ║  │   └─▶ Prompt Chaining ⭐⭐⭐                                   ║\n# ║  │                                                               ║\n# ║  ├─▶ Need to explore multiple approaches?                       ║\n# ║  │   └─▶ Tree-of-Thought ⭐⭐⭐⭐⭐                                ║\n# ║  │                                                               ║\n# ║  ├─▶ Want to improve quality iteratively?                       ║\n# ║  │   ├─▶ General improvement → Reflection ⭐⭐⭐⭐                 ║\n# ║  │   ├─▶ Criteria-based → Self-Feedback ⭐⭐⭐⭐                   ║\n# ║  │   └─▶ Find all flaws → Self-Critique ⭐⭐⭐⭐⭐                 ║\n# ║  │                                                               ║\n# ║  └─▶ Need evaluation?                                           ║\n# ║      ├─▶ Single output → LLM-as-Judge                           ║\n# ║      ├─▶ Compare options → Pairwise Comparison                  ║\n# ║      ├─▶ No reference → Reference-free                          ║\n# ║      ├─▶ With reference → BLEU/ROUGE                            ║\n# ║      ├─▶ Check facts → Hallucination Detection                  ║\n# ║      └─▶ Multiple perspectives → Multi-Agent Debate             ║\n# ╚══════════════════════════════════════════════════════════════════╝\n# ```\n# \n# ## 📊 เทคนิคและเมื่อไหร่ควรใช้:\n# \n# | เทคนิค | ใช้เมื่อ | ความซับซ้อน | API Calls | Cost |\n# |--------|---------|------------|-----------|------|\n# | Few-shot | ต้องการ format เฉพาะ | ⭐ | 1 | ต่ำ |\n# | Chain-of-Thought | ปัญหาต้องใช้เหตุผล | ⭐⭐ | 1 | ต่ำ |\n# | Self-Consistency | ต้องการความมั่นใจ | ⭐⭐⭐ | N (3-10) | กลาง |\n# | Prompt Chaining | งานซับซ้อนหลายขั้นตอน | ⭐⭐⭐ | K steps | กลาง |\n# | Reflection | ต้องการปรับปรุงคำตอบ | ⭐⭐⭐⭐ | 3 | กลาง |\n# | Tree-of-Thought | ปัญหามีหลายทางเลือก | ⭐⭐⭐⭐⭐ | 3-5 | สูง |\n# | Self-Feedback | ต้องการประเมินตามเกณฑ์ | ⭐⭐⭐⭐ | 3 | กลาง |\n# | Self-Critique | ต้องการหาจุดอ่อน | ⭐⭐⭐⭐⭐ | 4-6 | สูง |\n# \n# ## 📊 Evaluation Methods:\n# \n# | วิธี | ต้องการ Reference? | Scalable? | Best For |\n# |------|-------------------|-----------|----------|\n# | LLM-as-a-Judge | ❌ | ✅ | General quality |\n# | Pairwise Comparison | ❌ | ✅ | Comparing 2 options |\n# | Reference-free | ❌ | ✅ | Open-ended tasks |\n# | BLEU | ✅ | ✅ | Translation |\n# | ROUGE | ✅ | ✅ | Summarization |\n# | Semantic Similarity | ❌ | ✅ | Meaning comparison |\n# | Hallucination Detection | ❌ | ✅ | Factual accuracy |\n# | Multi-Agent Debate | ❌ | ⚠️ | Diverse perspectives |\n# | Human Evaluation | ❌ | ❌ | Gold standard |\n\n#%% [markdown]\n# ---\n# # 🏁 End of Lab\n# \n# ## 📚 แหล่งเรียนรู้เพิ่มเติม:\n# \n# ### Core Papers:\n# - [Brown et al. (2020) - \"Language Models are Few-Shot Learners\"](https://arxiv.org/abs/2005.14165)\n# - [Wei et al. (2022) - \"Chain-of-Thought Prompting\"](https://arxiv.org/abs/2201.11903)\n# - [Wang et al. (2023) - \"Self-Consistency Improves CoT\"](https://arxiv.org/abs/2203.11171)\n# - [Yao et al. (2023) - \"Tree of Thoughts\"](https://arxiv.org/abs/2305.10601)\n# - [Madaan et al. (2023) - \"Self-Refine\"](https://arxiv.org/abs/2303.17651)\n# - [Shinn et al. (2023) - \"Reflexion\"](https://arxiv.org/abs/2303.11366)\n# - [Zheng et al. (2023) - \"Judging LLM-as-a-Judge\"](https://arxiv.org/abs/2306.05685)\n# \n# ### Guides & Documentation:\n# - [Google AI Documentation](https://ai.google.dev/)\n# - [Prompt Engineering Guide](https://www.promptingguide.ai/)\n# - [Anthropic Prompt Engineering Guide](https://docs.anthropic.com/claude/docs/prompt-engineering)\n# \n# ## ✅ สิ่งที่ได้เรียนรู้:\n# 1. ✅ เทคนิค Prompt Engineering 8 เทคนิค พร้อมทฤษฎีเชิงลึก\n# 2. ✅ วิธี Evaluation 8+ วิธี (LLM-based + Automated Metrics)\n# 3. ✅ การเลือกเทคนิคที่เหมาะสมกับปัญหา (Decision Tree)\n# 4. ✅ การรวมเทคนิคเพื่อแก้ปัญหาซับซ้อน (Final Challenge)\n# 5. ✅ Bias awareness ใน LLM evaluation\n# 6. ✅ Best practices สำหรับ production use\n# \n# **Happy Prompting! 🎉**\n```\n\n---\n\n## 📋 สรุป Lab Structure (Enhanced)\n\n| Part | หัวข้อ | ระดับ | Theory | Infographic | Evaluation |\n|------|--------|-------|--------|-------------|------------|\n| 0 | Setup & Fundamentals | 🟢 | API & LLM Pipeline | ✅ Temperature, Architecture | - |\n| 1 | Few-shot Prompting | ⭐ | Shot types, Design principles | ✅ Shot Spectrum | Sensitivity Analysis |\n| 2 | Chain-of-Thought | ⭐⭐ | CoT mechanism, Trigger phrases | ✅ CoT vs Standard | When to use |\n| 3 | Self-Consistency CoT | ⭐⭐⭐ | Voting strategies, Parameters | ✅ SC mechanism | Consistency Score |\n| 4 | Prompt Chaining | ⭐⭐⭐ | Chaining patterns, Benefits | ✅ 4 patterns | Per-step validation |\n| 5 | Reflection | ⭐⭐⭐⭐ | Reflection cycle, Research | ✅ Cycle diagram | Iteration tracking |\n| 6 | Tree-of-Thought | ⭐⭐⭐⭐⭐ | ToT architecture, BFS/DFS | ✅ Tree structure | Branch scoring |\n| 7 | Self-Feedback | ⭐⭐⭐⭐ | Criteria-based assessment | ✅ vs Reflection | Score progression |\n| 8 | Self-Critique | ⭐⭐⭐⭐⭐ | Intensity spectrum, Checklist | ✅ Spectrum diagram | Adversarial rounds |\n| 9 | Evaluation Methods (Extended) | ⭐⭐⭐ | Taxonomy, Biases | ✅ Full taxonomy | **8 methods** |\n| 10 | Comprehensive Framework | ⭐⭐⭐⭐ | Architecture | ✅ Pipeline diagram | Combined report |\n| 11 | Final Challenge | ⭐⭐⭐⭐⭐ | Decision Tree | ✅ Challenge pipeline | Full pipeline |\n\n### 🆕 New Evaluation Techniques Added:\n1. **BLEU Score** — n-gram precision (translation/generation)\n2. **ROUGE Score** — ROUGE-1, ROUGE-2, ROUGE-L (summarization)\n3. **Semantic Similarity** — LLM-based meaning comparison\n4. **Hallucination Detection** — Dedicated fact-checking\n5. **Multi-Agent Debate** — Multiple persona evaluation\n6. **Position-Debiased Pairwise** — Bias-mitigated comparison\n7. **Sensitivity Analysis** — Few-shot example count impact\n8. **Comprehensive Report Generation** — Aggregated quality report"
    }
  ],
 
}

In [13]:
# Simple extraction
text = response["content"][0]["text"]
print(text)

# Save to file
with open("output.txt", "w", encoding="utf-8") as f:
    f.write(text)

# 🧪 Prompt Engineering Lab with Gemini API — Enhanced Edition

## 📚 Lab Overview & Learning Objectives

Lab นี้ออกแบบมาเพื่อให้ผู้เรียนได้ฝึกฝนเทคนิค Prompt Engineering ขั้นสูงผ่าน Gemini API โดยเริ่มจากพื้นฐานไปจนถึงเทคนิคขั้นสูง พร้อมระบบ Evaluation ที่ครบถ้วน

---

## 🗺️ Lab Roadmap — ภาพรวมเส้นทางการเรียนรู้

```
╔══════════════════════════════════════════════════════════════════════════════════╗
║                    🗺️ PROMPT ENGINEERING LEARNING PATH                         ║
╠══════════════════════════════════════════════════════════════════════════════════╣
║                                                                                ║
║  🟢 FOUNDATION (Part 0-2)                                                      ║
║  ┌─────────────┐    ┌─────────────────┐    ┌──────────────────┐                ║
║  │  Part 0     │───▶│   Part 1        │───▶│   Part 2         │                ║
║  │  Setup &    │    │   Few-shot      │    │   Chain-of-      │                ║
║  │  Connecti